
<a id='whatsnew-0230'></a>

# What’s new in 0.23.0 (May 15, 2018)

{{ header }}

This is a major release from 0.22.0 and includes a number of API changes,
deprecations, new features, enhancements, and performance improvements along
with a large number of bug fixes. We recommend that all users upgrade to this
version.

Highlights include:

- [Round-trippable JSON format with ‘table’ orient](#whatsnew-0230-enhancements-round-trippable-json).  
- [Instantiation from dicts respects order for Python 3.6+](#whatsnew-0230-api-breaking-dict-insertion-order).  
- [Dependent column arguments for assign](#whatsnew-0230-enhancements-assign-dependent).  
- [Merging / sorting on a combination of columns and index levels](#whatsnew-0230-enhancements-merge-on-columns-and-levels).  
- [Extending pandas with custom types](#whatsnew-023-enhancements-extension).  
- [Excluding unobserved categories from groupby](#whatsnew-0230-enhancements-categorical-grouping).  
- [Changes to make output shape of DataFrame.apply consistent](#whatsnew-0230-api-breaking-apply).  


Check the [API Changes](#whatsnew-0230-api-breaking) and [deprecations](#whatsnew-0230-deprecations) before updating.

Starting January 1, 2019, pandas feature releases will support Python 3 only.
See [Plan for dropping Python 2.7](install.ipynb#install-dropping-27) for more.

## What’s new in v0.23.0

- [New features](#New-features)  
  - [JSON read/write round-trippable with `orient='table'`](#JSON-read/write-round-trippable-with-`orient='table'`)  
  - [`.assign()` accepts dependent arguments](#`.assign()`-accepts-dependent-arguments)  
  - [Merging on a combination of columns and index levels](#Merging-on-a-combination-of-columns-and-index-levels)  
  - [Sorting by a combination of columns and index levels](#Sorting-by-a-combination-of-columns-and-index-levels)  
  - [Extending pandas with custom types (experimental)](#Extending-pandas-with-custom-types-(experimental))  
  - [New `observed` keyword for excluding unobserved categories in `groupby`](#New-`observed`-keyword-for-excluding-unobserved-categories-in-`groupby`)  
  - [Rolling/Expanding.apply() accepts `raw=False` to pass a `Series` to the function](#Rolling/Expanding.apply()-accepts-`raw=False`-to-pass-a-`Series`-to-the-function)  
  - [`DataFrame.interpolate` has gained the `limit_area` kwarg](#`DataFrame.interpolate`-has-gained-the-`limit_area`-kwarg)  
  - [`get_dummies` now supports `dtype` argument](#`get_dummies`-now-supports-`dtype`-argument)  
  - [Timedelta mod method](#Timedelta-mod-method)  
  - [`.rank()` handles `inf` values when `NaN` are present](#`.rank()`-handles-`inf`-values-when-`NaN`-are-present)  
  - [`Series.str.cat` has gained the `join` kwarg](#`Series.str.cat`-has-gained-the-`join`-kwarg)  
  - [`DataFrame.astype` performs column-wise conversion to `Categorical`](#`DataFrame.astype`-performs-column-wise-conversion-to-`Categorical`)  
  - [Other enhancements](#Other-enhancements)  
- [Backwards incompatible API changes](#Backwards-incompatible-API-changes)  
  - [Dependencies have increased minimum versions](#Dependencies-have-increased-minimum-versions)  
  - [Instantiation from dicts preserves dict insertion order for python 3.6+](#Instantiation-from-dicts-preserves-dict-insertion-order-for-python-3.6+)  
  - [Deprecate Panel](#Deprecate-Panel)  
  - [pandas.core.common removals](#pandas.core.common-removals)  
  - [Changes to make output of `DataFrame.apply` consistent](#Changes-to-make-output-of-`DataFrame.apply`-consistent)  
  - [Concatenation will no longer sort](#Concatenation-will-no-longer-sort)  
  - [Build changes](#Build-changes)  
  - [Index division by zero fills correctly](#Index-division-by-zero-fills-correctly)  
  - [Extraction of matching patterns from strings](#Extraction-of-matching-patterns-from-strings)  
  - [Default value for the `ordered` parameter of `CategoricalDtype`](#Default-value-for-the-`ordered`-parameter-of-`CategoricalDtype`)  
  - [Better pretty-printing of DataFrames in a terminal](#Better-pretty-printing-of-DataFrames-in-a-terminal)  
  - [Datetimelike API changes](#Datetimelike-API-changes)  
  - [Other API changes](#Other-API-changes)  
- [Deprecations](#Deprecations)  
- [Removal of prior version deprecations/changes](#Removal-of-prior-version-deprecations/changes)  
- [Performance improvements](#Performance-improvements)  
- [Documentation changes](#Documentation-changes)  
- [Bug fixes](#Bug-fixes)  
  - [Categorical](#Categorical)  
  - [Datetimelike](#Datetimelike)  
  - [Timedelta](#Timedelta)  
  - [Timezones](#Timezones)  
  - [Offsets](#Offsets)  
  - [Numeric](#Numeric)  
  - [Strings](#Strings)  
  - [Indexing](#Indexing)  
  - [MultiIndex](#MultiIndex)  
  - [I/O](#I/O)  
  - [Plotting](#Plotting)  
  - [Groupby/resample/rolling](#Groupby/resample/rolling)  
  - [Sparse](#Sparse)  
  - [Reshaping](#Reshaping)  
  - [Other](#Other)  
- [Contributors](#Contributors)  


<a id='whatsnew-0230-enhancements'></a>

## New features


<a id='whatsnew-0230-enhancements-round-trippable-json'></a>

### JSON read/write round-trippable with `orient='table'`

A `DataFrame` can now be written to and subsequently read back via JSON while preserving metadata through usage of the `orient='table'` argument (see [GH18912](https://github.com/pandas-dev/pandas/issues/18912) and [GH9146](https://github.com/pandas-dev/pandas/issues/9146)). Previously, none of the available `orient` values guaranteed the preservation of dtypes and index names, amongst other metadata.

Please note that the string index is not supported with the round trip format, as it is used by default in `write_json` to indicate a missing index name.


<a id='whatsnew-0230-enhancements-assign-dependent'></a>

### `.assign()` accepts dependent arguments

The `DataFrame.assign()` now accepts dependent keyword arguments for python version later than 3.6 (see also [PEP 468](https://www.python.org/dev/peps/pep-0468/)). Later keyword arguments may now refer to earlier ones if the argument is a callable. See the
[documentation here](getting_started/dsintro.ipynb#dsintro-chained-assignment) ([GH14207](https://github.com/pandas-dev/pandas/issues/14207))

This may subtly change the behavior of your code when you’re
using `.assign()` to update an existing column. Previously, callables
referring to other variables being updated would get the “old” values

Previous behavior:

```ipython
In [2]: df = pd.DataFrame({"A": [1, 2, 3]})

In [3]: df.assign(A=lambda df: df.A + 1, C=lambda df: df.A * -1)
Out[3]:
   A  C
0  2 -1
1  3 -2
2  4 -3
```


New behavior:


<a id='whatsnew-0230-enhancements-merge-on-columns-and-levels'></a>

### Merging on a combination of columns and index levels

Strings passed to `DataFrame.merge()` as the `on`, `left_on`, and `right_on`
parameters may now refer to either column names or index level names.
This enables merging `DataFrame` instances on a combination of index levels
and columns without resetting indexes. See the [Merge on columns and
levels](user_guide/merging.ipynb#merging-merge-on-columns-and-levels) documentation section.
([GH14355](https://github.com/pandas-dev/pandas/issues/14355))


<a id='whatsnew-0230-enhancements-sort-by-columns-and-levels'></a>

### Sorting by a combination of columns and index levels

Strings passed to `DataFrame.sort_values()` as the `by` parameter may
now refer to either column names or index level names.  This enables sorting
`DataFrame` instances by a combination of index levels and columns without
resetting indexes. See the [Sorting by Indexes and Values](getting_started/basics.ipynb#basics-sort-indexes-and-values) documentation section.
([GH14353](https://github.com/pandas-dev/pandas/issues/14353))


<a id='whatsnew-023-enhancements-extension'></a>

### Extending pandas with custom types (experimental)

Pandas now supports storing array-like objects that aren’t necessarily 1-D NumPy
arrays as columns in a DataFrame or values in a Series. This allows third-party
libraries to implement extensions to NumPy’s types, similar to how pandas
implemented categoricals, datetimes with timezones, periods, and intervals.

As a demonstration, we’ll use [cyberpandas](https://cyberpandas.readthedocs.io/en/latest/), which provides an `IPArray` type
for storing ip addresses.

```ipython
In [1]: from cyberpandas import IPArray

In [2]: values = IPArray([
   ...:     0,
   ...:     3232235777,
   ...:     42540766452641154071740215577757643572
   ...: ])
   ...:
   ...:
```


`IPArray` isn’t a normal 1-D NumPy array, but because it’s a pandas
`ExtensionArray`, it can be stored properly inside pandas’ containers.

```ipython
In [3]: ser = pd.Series(values)

In [4]: ser
Out[4]:
0                         0.0.0.0
1                     192.168.1.1
2    2001:db8:85a3::8a2e:370:7334
dtype: ip
```


Notice that the dtype is `ip`. The missing value semantics of the underlying
array are respected:

```ipython
In [5]: ser.isna()
Out[5]:
0     True
1    False
2    False
dtype: bool
```


For more, see the [extension types](development/extending.ipynb#extending-extension-types)
documentation. If you build an extension array, publicize it on our
[ecosystem page](ecosystem.ipynb#ecosystem-extensions).


<a id='whatsnew-0230-enhancements-categorical-grouping'></a>

### New `observed` keyword for excluding unobserved categories in `groupby`

Grouping by a categorical includes the unobserved categories in the output.
When grouping by multiple categorical columns, this means you get the cartesian product of all the
categories, including combinations where there are no observations, which can result in a large
number of groups. We have added a keyword `observed` to control this behavior, it defaults to
`observed=False` for backward-compatibility. ([GH14942](https://github.com/pandas-dev/pandas/issues/14942), [GH8138](https://github.com/pandas-dev/pandas/issues/8138), [GH15217](https://github.com/pandas-dev/pandas/issues/15217), [GH17594](https://github.com/pandas-dev/pandas/issues/17594), [GH8669](https://github.com/pandas-dev/pandas/issues/8669), [GH20583](https://github.com/pandas-dev/pandas/issues/20583), [GH20902](https://github.com/pandas-dev/pandas/issues/20902))

To show all values, the previous behavior:

To show only observed values:

For pivoting operations, this behavior is *already* controlled by the `dropna` keyword:


<a id='whatsnew-0230-enhancements-window-raw'></a>

### Rolling/Expanding.apply() accepts `raw=False` to pass a `Series` to the function

`Series.rolling().apply()`, `DataFrame.rolling().apply()`,
`Series.expanding().apply()`, and `DataFrame.expanding().apply()` have gained a `raw=None` parameter.
This is similar to `DataFame.apply()`. This parameter, if `True` allows one to send a `np.ndarray` to the applied function. If `False` a `Series` will be passed. The
default is `None`, which preserves backward compatibility, so this will default to `True`, sending an `np.ndarray`.
In a future version the default will be changed to `False`, sending a `Series`. ([GH5071](https://github.com/pandas-dev/pandas/issues/5071), [GH20584](https://github.com/pandas-dev/pandas/issues/20584))

Pass a `Series`:

Mimic the original behavior of passing a ndarray:


<a id='whatsnew-0210-enhancements-limit-area'></a>

### `DataFrame.interpolate` has gained the `limit_area` kwarg

`DataFrame.interpolate()` has gained a `limit_area` parameter to allow further control of which `NaN` s are replaced.
Use `limit_area='inside'` to fill only NaNs surrounded by valid values or use `limit_area='outside'` to fill only `NaN` s
outside the existing valid values while preserving those inside.  ([GH16284](https://github.com/pandas-dev/pandas/issues/16284)) See the [full documentation here](user_guide/missing_data.ipynb#missing-data-interp-limits).

Fill one consecutive inside value in both directions

Fill all consecutive outside values backward

Fill all consecutive outside values in both directions


<a id='whatsnew-0210-enhancements-get-dummies-dtype'></a>

### `get_dummies` now supports `dtype` argument

The `get_dummies()` now accepts a `dtype` argument, which specifies a dtype for the new columns. The default remains uint8. ([GH18330](https://github.com/pandas-dev/pandas/issues/18330))


<a id='whatsnew-0230-enhancements-timedelta-mod'></a>

### Timedelta mod method

`mod` (%) and `divmod` operations are now defined on `Timedelta` objects
when operating with either timedelta-like or with numeric arguments.
See the [documentation here](user_guide/timedeltas.ipynb#timedeltas-mod-divmod). ([GH19365](https://github.com/pandas-dev/pandas/issues/19365))


<a id='whatsnew-0230-enhancements-ran-inf'></a>

### `.rank()` handles `inf` values when `NaN` are present

In previous versions, `.rank()` would assign `inf` elements `NaN` as their ranks. Now ranks are calculated properly. ([GH6945](https://github.com/pandas-dev/pandas/issues/6945))

Previous behavior:

```ipython
In [11]: s.rank()
Out[11]:
0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
dtype: float64
```


Current behavior:

Furthermore, previously if you rank `inf` or `-inf` values together with `NaN` values, the calculation won’t distinguish `NaN` from infinity when using ‘top’ or ‘bottom’ argument.

Previous behavior:

```ipython
In [15]: s.rank(na_option='top')
Out[15]:
0    2.5
1    2.5
2    2.5
3    2.5
dtype: float64
```


Current behavior:

These bugs were squashed:

- Bug in `DataFrame.rank()` and `Series.rank()` when `method='dense'` and `pct=True` in which percentile ranks were not being used with the number of distinct observations ([GH15630](https://github.com/pandas-dev/pandas/issues/15630))  
- Bug in `Series.rank()` and `DataFrame.rank()` when `ascending='False'` failed to return correct ranks for infinity if `NaN` were present ([GH19538](https://github.com/pandas-dev/pandas/issues/19538))  
- Bug in `DataFrameGroupBy.rank()` where ranks were incorrect when both infinity and `NaN` were present ([GH20561](https://github.com/pandas-dev/pandas/issues/20561))  



<a id='whatsnew-0230-enhancements-str-cat-align'></a>

### `Series.str.cat` has gained the `join` kwarg

Previously, `Series.str.cat()` did not – in contrast to most of `pandas` – align `Series` on their index before concatenation (see [GH18657](https://github.com/pandas-dev/pandas/issues/18657)).
The method has now gained a keyword `join` to control the manner of alignment, see examples below and [here](user_guide/text.ipynb#text-concatenate).

In v.0.23 join will default to None (meaning no alignment), but this default will change to `'left'` in a future version of pandas.

Furthermore, `Series.str.cat()` now works for `CategoricalIndex` as well (previously raised a `ValueError`; see [GH20842](https://github.com/pandas-dev/pandas/issues/20842)).


<a id='whatsnew-0230-enhancements-astype-category'></a>

### `DataFrame.astype` performs column-wise conversion to `Categorical`

`DataFrame.astype()` can now perform column-wise conversion to `Categorical` by supplying the string `'category'` or
a `CategoricalDtype`. Previously, attempting this would raise a `NotImplementedError`. See the
[Object creation](user_guide/categorical.ipynb#categorical-objectcreation) section of the documentation for more details and examples. ([GH12860](https://github.com/pandas-dev/pandas/issues/12860), [GH18099](https://github.com/pandas-dev/pandas/issues/18099))

Supplying the string `'category'` performs column-wise conversion, with only labels appearing in a given column set as categories:

Supplying a `CategoricalDtype` will make the categories in each column consistent with the supplied dtype:


<a id='whatsnew-0230-enhancements-other'></a>

### Other enhancements

- Unary `+` now permitted for `Series` and `DataFrame` as  numeric operator ([GH16073](https://github.com/pandas-dev/pandas/issues/16073))  
- Better support for `to_excel()` output with the `xlsxwriter` engine. ([GH16149](https://github.com/pandas-dev/pandas/issues/16149))  
- `pandas.tseries.frequencies.to_offset()` now accepts leading ‘+’ signs e.g. ‘+1h’. ([GH18171](https://github.com/pandas-dev/pandas/issues/18171))  
- `MultiIndex.unique()` now supports the `level=` argument, to get unique values from a specific index level ([GH17896](https://github.com/pandas-dev/pandas/issues/17896))  
- `pandas.io.formats.style.Styler` now has method `hide_index()` to determine whether the index will be rendered in output ([GH14194](https://github.com/pandas-dev/pandas/issues/14194))  
- `pandas.io.formats.style.Styler` now has method `hide_columns()` to determine whether columns will be hidden in output ([GH14194](https://github.com/pandas-dev/pandas/issues/14194))  
- Improved wording of `ValueError` raised in `to_datetime()` when `unit=` is passed with a non-convertible value ([GH14350](https://github.com/pandas-dev/pandas/issues/14350))  
- `Series.fillna()` now accepts a Series or a dict as a `value` for a categorical dtype ([GH17033](https://github.com/pandas-dev/pandas/issues/17033))  
- `pandas.read_clipboard()` updated to use qtpy, falling back to PyQt5 and then PyQt4, adding compatibility with Python3 and multiple python-qt bindings ([GH17722](https://github.com/pandas-dev/pandas/issues/17722))  
- Improved wording of `ValueError` raised in `read_csv()` when the `usecols` argument cannot match all columns. ([GH17301](https://github.com/pandas-dev/pandas/issues/17301))  
- `DataFrame.corrwith()` now silently drops non-numeric columns when passed a Series. Before, an exception was raised ([GH18570](https://github.com/pandas-dev/pandas/issues/18570)).  
- `IntervalIndex` now supports time zone aware `Interval` objects ([GH18537](https://github.com/pandas-dev/pandas/issues/18537), [GH18538](https://github.com/pandas-dev/pandas/issues/18538))  
- `Series()` / `DataFrame()` tab completion also returns identifiers in the first level of a `MultiIndex()`. ([GH16326](https://github.com/pandas-dev/pandas/issues/16326))  
- `read_excel()` has gained the `nrows` parameter ([GH16645](https://github.com/pandas-dev/pandas/issues/16645))  
- `DataFrame.append()` can now in more cases preserve the type of the calling dataframe’s columns (e.g. if both are `CategoricalIndex`) ([GH18359](https://github.com/pandas-dev/pandas/issues/18359))  
- `DataFrame.to_json()` and `Series.to_json()` now accept an `index` argument which allows the user to exclude the index from the JSON output ([GH17394](https://github.com/pandas-dev/pandas/issues/17394))  
- `IntervalIndex.to_tuples()` has gained the `na_tuple` parameter to control whether NA is returned as a tuple of NA, or NA itself ([GH18756](https://github.com/pandas-dev/pandas/issues/18756))  
- `Categorical.rename_categories`, `CategoricalIndex.rename_categories` and `Series.cat.rename_categories`
  can now take a callable as their argument ([GH18862](https://github.com/pandas-dev/pandas/issues/18862))  
- `Interval` and `IntervalIndex` have gained a `length` attribute ([GH18789](https://github.com/pandas-dev/pandas/issues/18789))  
- `Resampler` objects now have a functioning `pipe` method.
  Previously, calls to `pipe` were diverted to  the `mean` method ([GH17905](https://github.com/pandas-dev/pandas/issues/17905)).  
- `is_scalar()` now returns `True` for `DateOffset` objects ([GH18943](https://github.com/pandas-dev/pandas/issues/18943)).  
- `DataFrame.pivot()` now accepts a list for the `values=` kwarg ([GH17160](https://github.com/pandas-dev/pandas/issues/17160)).  
- Added `pandas.api.extensions.register_dataframe_accessor()`,
  `pandas.api.extensions.register_series_accessor()`, and
  `pandas.api.extensions.register_index_accessor()`, accessor for libraries downstream of pandas
  to register custom accessors like `.cat` on pandas objects. See
  [Registering Custom Accessors](development/extending.ipynb#extending-register-accessors) for more ([GH14781](https://github.com/pandas-dev/pandas/issues/14781)).  
- `IntervalIndex.astype` now supports conversions between subtypes when passed an `IntervalDtype` ([GH19197](https://github.com/pandas-dev/pandas/issues/19197))  
- `IntervalIndex` and its associated constructor methods (`from_arrays`, `from_breaks`, `from_tuples`) have gained a `dtype` parameter ([GH19262](https://github.com/pandas-dev/pandas/issues/19262))  
- Added `pandas.core.groupby.SeriesGroupBy.is_monotonic_increasing()` and `pandas.core.groupby.SeriesGroupBy.is_monotonic_decreasing()` ([GH17015](https://github.com/pandas-dev/pandas/issues/17015))  
- For subclassed `DataFrames`, `DataFrame.apply()` will now preserve the `Series` subclass (if defined) when passing the data to the applied function ([GH19822](https://github.com/pandas-dev/pandas/issues/19822))  
- `DataFrame.from_dict()` now accepts a `columns` argument that can be used to specify the column names when `orient='index'` is used ([GH18529](https://github.com/pandas-dev/pandas/issues/18529))  
- Added option `display.html.use_mathjax` so [MathJax](https://www.mathjax.org/) can be disabled when rendering tables in `Jupyter` notebooks ([GH19856](https://github.com/pandas-dev/pandas/issues/19856), [GH19824](https://github.com/pandas-dev/pandas/issues/19824))  
- `DataFrame.replace()` now supports the `method` parameter, which can be used to specify the replacement method when `to_replace` is a scalar, list or tuple and `value` is `None` ([GH19632](https://github.com/pandas-dev/pandas/issues/19632))  
- `Timestamp.month_name()`, `DatetimeIndex.month_name()`, and `Series.dt.month_name()` are now available ([GH12805](https://github.com/pandas-dev/pandas/issues/12805))  
- `Timestamp.day_name()` and `DatetimeIndex.day_name()` are now available to return day names with a specified locale ([GH12806](https://github.com/pandas-dev/pandas/issues/12806))  
- `DataFrame.to_sql()` now performs a multi-value insert if the underlying connection supports itk rather than inserting row by row.
  `SQLAlchemy` dialects supporting multi-value inserts include: `mysql`, `postgresql`, `sqlite` and any dialect with `supports_multivalues_insert`. ([GH14315](https://github.com/pandas-dev/pandas/issues/14315), [GH8953](https://github.com/pandas-dev/pandas/issues/8953))  
- `read_html()` now accepts a `displayed_only` keyword argument to controls whether or not hidden elements are parsed (`True` by default) ([GH20027](https://github.com/pandas-dev/pandas/issues/20027))  
- `read_html()` now reads all `<tbody>` elements in a `<table>`, not just the first. ([GH20690](https://github.com/pandas-dev/pandas/issues/20690))  
- `quantile()` and `quantile()` now accept the `interpolation` keyword, `linear` by default ([GH20497](https://github.com/pandas-dev/pandas/issues/20497))  
- zip compression is supported via `compression=zip` in `DataFrame.to_pickle()`, `Series.to_pickle()`, `DataFrame.to_csv()`, `Series.to_csv()`, `DataFrame.to_json()`, `Series.to_json()`. ([GH17778](https://github.com/pandas-dev/pandas/issues/17778))  
- `WeekOfMonth` constructor now supports `n=0` ([GH20517](https://github.com/pandas-dev/pandas/issues/20517)).  
- `DataFrame` and `Series` now support matrix multiplication (`@`) operator ([GH10259](https://github.com/pandas-dev/pandas/issues/10259)) for Python>=3.5  
- Updated `DataFrame.to_gbq()` and `pandas.read_gbq()` signature and documentation to reflect changes from
  the Pandas-GBQ library version 0.4.0. Adds intersphinx mapping to Pandas-GBQ
  library. ([GH20564](https://github.com/pandas-dev/pandas/issues/20564))  
- Added new writer for exporting Stata dta files in version 117, `StataWriter117`.  This format supports exporting strings with lengths up to 2,000,000 characters ([GH16450](https://github.com/pandas-dev/pandas/issues/16450))  
- `to_hdf()` and `read_hdf()` now accept an `errors` keyword argument to control encoding error handling ([GH20835](https://github.com/pandas-dev/pandas/issues/20835))  
- `cut()` has gained the `duplicates='raise'|'drop'` option to control whether to raise on duplicated edges ([GH20947](https://github.com/pandas-dev/pandas/issues/20947))  
- `date_range()`, `timedelta_range()`, and `interval_range()` now return a linearly spaced index if `start`, `stop`, and `periods` are specified, but `freq` is not. ([GH20808](https://github.com/pandas-dev/pandas/issues/20808), [GH20983](https://github.com/pandas-dev/pandas/issues/20983), [GH20976](https://github.com/pandas-dev/pandas/issues/20976))  



<a id='whatsnew-0230-api-breaking'></a>

## Backwards incompatible API changes


<a id='whatsnew-0230-api-breaking-deps'></a>

### Dependencies have increased minimum versions

We have updated our minimum supported versions of dependencies ([GH15184](https://github.com/pandas-dev/pandas/issues/15184)).
If installed, we now require:

[](https://github.com/pandas-dev/pandas/issues/15184)[](https://github.com/pandas-dev/pandas/issues/15184)[](https://github.com/pandas-dev/pandas/issues/20082)[](https://github.com/pandas-dev/pandas/issues/20698)|Package|Minimum Version|Required|Issue|
|:---------------:|:---------------:|:--------:|:-------------:|
|python-dateutil|2.5.0|X|GH15184|
|openpyxl|2.4.0||GH15184|
|beautifulsoup4|4.2.1||GH20082|
|setuptools|24.2.0||GH20698|

<a id='whatsnew-0230-api-breaking-dict-insertion-order'></a>

### Instantiation from dicts preserves dict insertion order for python 3.6+

Until Python 3.6, dicts in Python had no formally defined ordering. For Python
version 3.6 and later, dicts are ordered by insertion order, see
[PEP 468](https://www.python.org/dev/peps/pep-0468/).
Pandas will use the dict’s insertion order, when creating a `Series` or
`DataFrame` from a dict and you’re using Python version 3.6 or
higher. ([GH19884](https://github.com/pandas-dev/pandas/issues/19884))

Previous behavior (and current behavior if on Python < 3.6):

```ipython
In [16]: pd.Series({'Income': 2000,
   ....:            'Expenses': -1500,
   ....:            'Taxes': -200,
   ....:            'Net result': 300})
Out[16]:
Expenses     -1500
Income        2000
Net result     300
Taxes         -200
dtype: int64
```


Note the Series above is ordered alphabetically by the index values.

New behavior (for Python >= 3.6):

Notice that the Series is now ordered by insertion order. This new behavior is
used for all relevant pandas types (`Series`, `DataFrame`, `SparseSeries`
and `SparseDataFrame`).

If you wish to retain the old behavior while using Python >= 3.6, you can use
`.sort_index()`:


<a id='whatsnew-0230-api-breaking-deprecate-panel'></a>

### Deprecate Panel

`Panel` was deprecated in the 0.20.x release, showing as a `DeprecationWarning`. Using `Panel` will now show a `FutureWarning`. The recommended way to represent 3-D data are
with a `MultiIndex` on a `DataFrame` via the `to_frame()` or with the [xarray package](http://xarray.pydata.org/en/stable/). Pandas
provides a `to_xarray()` method to automate this conversion ([GH13563](https://github.com/pandas-dev/pandas/issues/13563), [GH18324](https://github.com/pandas-dev/pandas/issues/18324)).

```ipython
In [75]: import pandas.util.testing as tm

In [76]: p = tm.makePanel()

In [77]: p
Out[77]:
<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 3 (major_axis) x 4 (minor_axis)
Items axis: ItemA to ItemC
Major_axis axis: 2000-01-03 00:00:00 to 2000-01-05 00:00:00
Minor_axis axis: A to D
```


Convert to a MultiIndex DataFrame

```ipython
In [78]: p.to_frame()
Out[78]:
                     ItemA     ItemB     ItemC
major      minor
2000-01-03 A      0.469112  0.721555  0.404705
           B     -1.135632  0.271860 -1.039268
           C      0.119209  0.276232 -1.344312
           D     -2.104569  0.113648 -0.109050
2000-01-04 A     -0.282863 -0.706771  0.577046
           B      1.212112 -0.424972 -0.370647
           C     -1.044236 -1.087401  0.844885
           D     -0.494929 -1.478427  1.643563
2000-01-05 A     -1.509059 -1.039575 -1.715002
           B     -0.173215  0.567020 -1.157892
           C     -0.861849 -0.673690  1.075770
           D      1.071804  0.524988 -1.469388

[12 rows x 3 columns]
```


Convert to an xarray DataArray

```ipython
In [79]: p.to_xarray()
Out[79]:
<xarray.DataArray (items: 3, major_axis: 3, minor_axis: 4)>
array([[[ 0.469112, -1.135632,  0.119209, -2.104569],
        [-0.282863,  1.212112, -1.044236, -0.494929],
        [-1.509059, -0.173215, -0.861849,  1.071804]],

       [[ 0.721555,  0.27186 ,  0.276232,  0.113648],
        [-0.706771, -0.424972, -1.087401, -1.478427],
        [-1.039575,  0.56702 , -0.67369 ,  0.524988]],

       [[ 0.404705, -1.039268, -1.344312, -0.10905 ],
        [ 0.577046, -0.370647,  0.844885,  1.643563],
        [-1.715002, -1.157892,  1.07577 , -1.469388]]])
Coordinates:
  * items       (items) object 'ItemA' 'ItemB' 'ItemC'
  * major_axis  (major_axis) datetime64[ns] 2000-01-03 2000-01-04 2000-01-05
  * minor_axis  (minor_axis) object 'A' 'B' 'C' 'D'
```



<a id='whatsnew-0230-api-breaking-core-common'></a>

### pandas.core.common removals

The following error & warning messages are removed from `pandas.core.common` ([GH13634](https://github.com/pandas-dev/pandas/issues/13634), [GH19769](https://github.com/pandas-dev/pandas/issues/19769)):

- `PerformanceWarning`  
- `UnsupportedFunctionCall`  
- `UnsortedIndexError`  
- `AbstractMethodError`  


These are available from import from `pandas.errors` (since 0.19.0).


<a id='whatsnew-0230-api-breaking-apply'></a>

### Changes to make output of `DataFrame.apply` consistent

`DataFrame.apply()` was inconsistent when applying an arbitrary user-defined-function that returned a list-like with `axis=1`. Several bugs and inconsistencies
are resolved. If the applied function returns a Series, then pandas will return a DataFrame; otherwise a Series will be returned, this includes the case
where a list-like (e.g. `tuple` or `list` is returned) ([GH16353](https://github.com/pandas-dev/pandas/issues/16353), [GH17437](https://github.com/pandas-dev/pandas/issues/17437), [GH17970](https://github.com/pandas-dev/pandas/issues/17970), [GH17348](https://github.com/pandas-dev/pandas/issues/17348), [GH17892](https://github.com/pandas-dev/pandas/issues/17892), [GH18573](https://github.com/pandas-dev/pandas/issues/18573),
[GH17602](https://github.com/pandas-dev/pandas/issues/17602), [GH18775](https://github.com/pandas-dev/pandas/issues/18775), [GH18901](https://github.com/pandas-dev/pandas/issues/18901), [GH18919](https://github.com/pandas-dev/pandas/issues/18919)).

Previous behavior: if the returned shape happened to match the length of original columns, this would return a `DataFrame`.
If the return shape did not match, a `Series` with lists was returned.

In [ ]:
In [3]: df.apply(lambda x: [1, 2, 3], axis=1)
Out[3]:
   A  B  C
0  1  2  3
1  1  2  3
2  1  2  3
3  1  2  3
4  1  2  3
5  1  2  3

In [4]: df.apply(lambda x: [1, 2], axis=1)
Out[4]:
0    [1, 2]
1    [1, 2]
2    [1, 2]
3    [1, 2]
4    [1, 2]
5    [1, 2]
dtype: object

New behavior: When the applied function returns a list-like, this will now *always* return a `Series`.

To have expanded columns, you can use `result_type='expand'`

To broadcast the result across the original columns (the old behaviour for
list-likes of the correct length), you can use `result_type='broadcast'`.
The shape must match the original columns.

Returning a `Series` allows one to control the exact return structure and column names:


<a id='whatsnew-0230-api-breaking-concat'></a>

### Concatenation will no longer sort

In a future version of pandas `pandas.concat()` will no longer sort the non-concatenation axis when it is not already aligned.
The current behavior is the same as the previous (sorting), but now a warning is issued when `sort` is not specified and the non-concatenation axis is not aligned ([GH4588](https://github.com/pandas-dev/pandas/issues/4588)).

To keep the previous behavior (sorting) and silence the warning, pass `sort=True`

To accept the future behavior (no sorting), pass `sort=False`

Note that this change also applies to `DataFrame.append()`, which has also received a `sort` keyword for controlling this behavior.


<a id='whatsnew-0230-api-breaking-build-changes'></a>

### Build changes

- Building pandas for development now requires `cython >= 0.24` ([GH18613](https://github.com/pandas-dev/pandas/issues/18613))  
- Building from source now explicitly requires `setuptools` in `setup.py` ([GH18113](https://github.com/pandas-dev/pandas/issues/18113))  
- Updated conda recipe to be in compliance with conda-build 3.0+ ([GH18002](https://github.com/pandas-dev/pandas/issues/18002))  



<a id='whatsnew-0230-api-breaking-index-division-by-zero'></a>

### Index division by zero fills correctly

Division operations on `Index` and subclasses will now fill division of positive numbers by zero with `np.inf`, division of negative numbers by zero with `-np.inf` and 0 / 0 with `np.nan`.  This matches existing `Series` behavior. ([GH19322](https://github.com/pandas-dev/pandas/issues/19322), [GH19347](https://github.com/pandas-dev/pandas/issues/19347))

Previous behavior:

```ipython
In [6]: index = pd.Int64Index([-1, 0, 1])

In [7]: index / 0
Out[7]: Int64Index([0, 0, 0], dtype='int64')

# Previous behavior yielded different results depending on the type of zero in the divisor
In [8]: index / 0.0
Out[8]: Float64Index([-inf, nan, inf], dtype='float64')

In [9]: index = pd.UInt64Index([0, 1])

In [10]: index / np.array([0, 0], dtype=np.uint64)
Out[10]: UInt64Index([0, 0], dtype='uint64')

In [11]: pd.RangeIndex(1, 5) / 0
ZeroDivisionError: integer division or modulo by zero
```


Current behavior:


<a id='whatsnew-0230-api-breaking-extract'></a>

### Extraction of matching patterns from strings

By default, extracting matching patterns from strings with `str.extract()` used to return a
`Series` if a single group was being extracted (a `DataFrame` if more than one group was
extracted). As of Pandas 0.23.0 `str.extract()` always returns a `DataFrame`, unless
`expand` is set to `False`. Finally, `None` was an accepted value for
the `expand` parameter (which was equivalent to `False`), but now raises a `ValueError`. ([GH11386](https://github.com/pandas-dev/pandas/issues/11386))

Previous behavior:

```ipython
In [1]: s = pd.Series(['number 10', '12 eggs'])

In [2]: extracted = s.str.extract(r'.*(\d\d).*')

In [3]: extracted
Out [3]:
0    10
1    12
dtype: object

In [4]: type(extracted)
Out [4]:
pandas.core.series.Series
```


New behavior:

To restore previous behavior, simply set `expand` to `False`:


<a id='whatsnew-0230-api-breaking-cdt-ordered'></a>

### Default value for the `ordered` parameter of `CategoricalDtype`

The default value of the `ordered` parameter for `CategoricalDtype` has changed from `False` to `None` to allow updating of `categories` without impacting `ordered`.  Behavior should remain consistent for downstream objects, such as `Categorical` ([GH18790](https://github.com/pandas-dev/pandas/issues/18790))

In previous versions, the default value for the `ordered` parameter was `False`.  This could potentially lead to the `ordered` parameter unintentionally being changed from `True` to `False` when users attempt to update `categories` if `ordered` is not explicitly specified, as it would silently default to `False`.  The new behavior for `ordered=None` is to retain the existing value of `ordered`.

New behavior:

```ipython
In [2]: from pandas.api.types import CategoricalDtype

In [3]: cat = pd.Categorical(list('abcaba'), ordered=True, categories=list('cba'))

In [4]: cat
Out[4]:
[a, b, c, a, b, a]
Categories (3, object): [c < b < a]

In [5]: cdt = CategoricalDtype(categories=list('cbad'))

In [6]: cat.astype(cdt)
Out[6]:
[a, b, c, a, b, a]
Categories (4, object): [c < b < a < d]
```


Notice in the example above that the converted `Categorical` has retained `ordered=True`.  Had the default value for `ordered` remained as `False`, the converted `Categorical` would have become unordered, despite `ordered=False` never being explicitly specified.  To change the value of `ordered`, explicitly pass it to the new dtype, e.g. `CategoricalDtype(categories=list('cbad'), ordered=False)`.

Note that the unintentional conversion of `ordered` discussed above did not arise in previous versions due to separate bugs that prevented `astype` from doing any type of category to category conversion ([GH10696](https://github.com/pandas-dev/pandas/issues/10696), [GH18593](https://github.com/pandas-dev/pandas/issues/18593)).  These bugs have been fixed in this release, and motivated changing the default value of `ordered`.


<a id='whatsnew-0230-api-breaking-pretty-printing'></a>

### Better pretty-printing of DataFrames in a terminal

Previously, the default value for the maximum number of columns was
`pd.options.display.max_columns=20`. This meant that relatively wide data
frames would not fit within the terminal width, and pandas would introduce line
breaks to display these 20 columns. This resulted in an output that was
relatively difficult to read:

<img src="whatsnew/../_static/print_df_old.png" style="">

If Python runs in a terminal, the maximum number of columns is now determined
automatically so that the printed data frame fits within the current terminal
width (`pd.options.display.max_columns=0`) ([GH17023](https://github.com/pandas-dev/pandas/issues/17023)). If Python runs
as a Jupyter kernel (such as the Jupyter QtConsole or a Jupyter notebook, as
well as in many IDEs), this value cannot be inferred automatically and is thus
set to 20 as in previous versions. In a terminal, this results in a much
nicer output:

<img src="whatsnew/../_static/print_df_new.png" style="">

Note that if you don’t like the new default, you can always set this option
yourself. To revert to the old setting, you can run this line:

In [ ]:
pd.options.display.max_columns = 20


<a id='whatsnew-0230-api-datetimelike'></a>

### Datetimelike API changes

- The default `Timedelta` constructor now accepts an `ISO 8601 Duration` string as an argument ([GH19040](https://github.com/pandas-dev/pandas/issues/19040))  
- Subtracting `NaT` from a `Series` with `dtype='datetime64[ns]'` returns a `Series` with `dtype='timedelta64[ns]'` instead of `dtype='datetime64[ns]'` ([GH18808](https://github.com/pandas-dev/pandas/issues/18808))  
- Addition or subtraction of `NaT` from `TimedeltaIndex` will return `TimedeltaIndex` instead of `DatetimeIndex` ([GH19124](https://github.com/pandas-dev/pandas/issues/19124))  
- `DatetimeIndex.shift()` and `TimedeltaIndex.shift()` will now raise `NullFrequencyError` (which subclasses `ValueError`, which was raised in older versions) when the index object frequency is `None` ([GH19147](https://github.com/pandas-dev/pandas/issues/19147))  
- Addition and subtraction of `NaN` from a `Series` with `dtype='timedelta64[ns]'` will raise a `TypeError` instead of treating the `NaN` as `NaT` ([GH19274](https://github.com/pandas-dev/pandas/issues/19274))  
- `NaT` division with [`datetime.timedelta`](https://docs.python.org/3/library/datetime.html#datetime.timedelta) will now return `NaN` instead of raising ([GH17876](https://github.com/pandas-dev/pandas/issues/17876))  
- Operations between a `Series` with dtype `dtype='datetime64[ns]'` and a `PeriodIndex` will correctly raises `TypeError` ([GH18850](https://github.com/pandas-dev/pandas/issues/18850))  
- Subtraction of `Series` with timezone-aware `dtype='datetime64[ns]'` with mis-matched timezones will raise `TypeError` instead of `ValueError` ([GH18817](https://github.com/pandas-dev/pandas/issues/18817))  
- `Timestamp` will no longer silently ignore unused or invalid `tz` or `tzinfo` keyword arguments ([GH17690](https://github.com/pandas-dev/pandas/issues/17690))  
- `Timestamp` will no longer silently ignore invalid `freq` arguments ([GH5168](https://github.com/pandas-dev/pandas/issues/5168))  
- `CacheableOffset` and `WeekDay` are no longer available in the `pandas.tseries.offsets` module ([GH17830](https://github.com/pandas-dev/pandas/issues/17830))  
- `pandas.tseries.frequencies.get_freq_group()` and `pandas.tseries.frequencies.DAYS` are removed from the public API ([GH18034](https://github.com/pandas-dev/pandas/issues/18034))  
- `Series.truncate()` and `DataFrame.truncate()` will raise a `ValueError` if the index is not sorted instead of an unhelpful `KeyError` ([GH17935](https://github.com/pandas-dev/pandas/issues/17935))  
- `Series.first` and `DataFrame.first` will now raise a `TypeError`
  rather than `NotImplementedError` when index is not a `DatetimeIndex` ([GH20725](https://github.com/pandas-dev/pandas/issues/20725)).  
- `Series.last` and `DataFrame.last` will now raise a `TypeError`
  rather than `NotImplementedError` when index is not a `DatetimeIndex` ([GH20725](https://github.com/pandas-dev/pandas/issues/20725)).  
- Restricted `DateOffset` keyword arguments. Previously, `DateOffset` subclasses allowed arbitrary keyword arguments which could lead to unexpected behavior. Now, only valid arguments will be accepted. ([GH17176](https://github.com/pandas-dev/pandas/issues/17176), [GH18226](https://github.com/pandas-dev/pandas/issues/18226)).  
- `pandas.merge()` provides a more informative error message when trying to merge on timezone-aware and timezone-naive columns ([GH15800](https://github.com/pandas-dev/pandas/issues/15800))  
- For `DatetimeIndex` and `TimedeltaIndex` with `freq=None`, addition or subtraction of integer-dtyped array or `Index` will raise `NullFrequencyError` instead of `TypeError` ([GH19895](https://github.com/pandas-dev/pandas/issues/19895))  
- `Timestamp` constructor now accepts a nanosecond keyword or positional argument ([GH18898](https://github.com/pandas-dev/pandas/issues/18898))  
- `DatetimeIndex` will now raise an `AttributeError` when the `tz` attribute is set after instantiation ([GH3746](https://github.com/pandas-dev/pandas/issues/3746))  
- `DatetimeIndex` with a `pytz` timezone will now return a consistent `pytz` timezone ([GH18595](https://github.com/pandas-dev/pandas/issues/18595))  



<a id='whatsnew-0230-api-other'></a>

### Other API changes

- `Series.astype()` and `Index.astype()` with an incompatible dtype will now raise a `TypeError` rather than a `ValueError` ([GH18231](https://github.com/pandas-dev/pandas/issues/18231))  
- `Series` construction with an `object` dtyped tz-aware datetime and `dtype=object` specified, will now return an `object` dtyped `Series`, previously this would infer the datetime dtype ([GH18231](https://github.com/pandas-dev/pandas/issues/18231))  
- A `Series` of `dtype=category` constructed from an empty `dict` will now have categories of `dtype=object` rather than `dtype=float64`, consistently with the case in which an empty list is passed ([GH18515](https://github.com/pandas-dev/pandas/issues/18515))  
- All-NaN levels in a `MultiIndex` are now assigned `float` rather than `object` dtype, promoting consistency with `Index` ([GH17929](https://github.com/pandas-dev/pandas/issues/17929)).  
- Levels names of a `MultiIndex` (when not None) are now required to be unique: trying to create a `MultiIndex` with repeated names will raise a `ValueError` ([GH18872](https://github.com/pandas-dev/pandas/issues/18872))  
- Both construction and renaming of `Index`/`MultiIndex` with non-hashable `name`/`names` will now raise `TypeError` ([GH20527](https://github.com/pandas-dev/pandas/issues/20527))  
- `Index.map()` can now accept `Series` and dictionary input objects ([GH12756](https://github.com/pandas-dev/pandas/issues/12756), [GH18482](https://github.com/pandas-dev/pandas/issues/18482), [GH18509](https://github.com/pandas-dev/pandas/issues/18509)).  
- `DataFrame.unstack()` will now default to filling with `np.nan` for `object` columns. ([GH12815](https://github.com/pandas-dev/pandas/issues/12815))  
- `IntervalIndex` constructor will raise if the `closed` parameter conflicts with how the input data is inferred to be closed ([GH18421](https://github.com/pandas-dev/pandas/issues/18421))  
- Inserting missing values into indexes will work for all types of indexes and automatically insert the correct type of missing value (`NaN`, `NaT`, etc.) regardless of the type passed in ([GH18295](https://github.com/pandas-dev/pandas/issues/18295))  
- When created with duplicate labels, `MultiIndex` now raises a `ValueError`. ([GH17464](https://github.com/pandas-dev/pandas/issues/17464))  
- `Series.fillna()` now raises a `TypeError` instead of a `ValueError` when passed a list, tuple or DataFrame as a `value` ([GH18293](https://github.com/pandas-dev/pandas/issues/18293))  
- `pandas.DataFrame.merge()` no longer casts a `float` column to `object` when merging on `int` and `float` columns ([GH16572](https://github.com/pandas-dev/pandas/issues/16572))  
- `pandas.merge()` now raises a `ValueError` when trying to merge on incompatible data types ([GH9780](https://github.com/pandas-dev/pandas/issues/9780))  
- The default NA value for `UInt64Index` has changed from 0 to `NaN`, which impacts methods that mask with NA, such as `UInt64Index.where()` ([GH18398](https://github.com/pandas-dev/pandas/issues/18398))  
- Refactored `setup.py` to use `find_packages` instead of explicitly listing out all subpackages ([GH18535](https://github.com/pandas-dev/pandas/issues/18535))  
- Rearranged the order of keyword arguments in `read_excel()` to align with `read_csv()` ([GH16672](https://github.com/pandas-dev/pandas/issues/16672))  
- `wide_to_long()` previously kept numeric-like suffixes as `object` dtype. Now they are cast to numeric if possible ([GH17627](https://github.com/pandas-dev/pandas/issues/17627))  
- In `read_excel()`, the `comment` argument is now exposed as a named parameter ([GH18735](https://github.com/pandas-dev/pandas/issues/18735))  
- Rearranged the order of keyword arguments in `read_excel()` to align with `read_csv()` ([GH16672](https://github.com/pandas-dev/pandas/issues/16672))  
- The options `html.border` and `mode.use_inf_as_null` were deprecated in prior versions, these will now show `FutureWarning` rather than a `DeprecationWarning` ([GH19003](https://github.com/pandas-dev/pandas/issues/19003))  
- `IntervalIndex` and `IntervalDtype` no longer support categorical, object, and string subtypes ([GH19016](https://github.com/pandas-dev/pandas/issues/19016))  
- `IntervalDtype` now returns `True` when compared against `'interval'` regardless of subtype, and `IntervalDtype.name` now returns `'interval'` regardless of subtype ([GH18980](https://github.com/pandas-dev/pandas/issues/18980))  
- `KeyError` now raises instead of `ValueError` in `drop()`, `drop()`, `drop()`, `drop()` when dropping a non-existent element in an axis with duplicates ([GH19186](https://github.com/pandas-dev/pandas/issues/19186))  
- `Series.to_csv()` now accepts a `compression` argument that works in the same way as the `compression` argument in `DataFrame.to_csv()` ([GH18958](https://github.com/pandas-dev/pandas/issues/18958))  
- Set operations (union, difference…) on `IntervalIndex` with incompatible index types will now raise a `TypeError` rather than a `ValueError` ([GH19329](https://github.com/pandas-dev/pandas/issues/19329))  
- `DateOffset` objects render more simply, e.g. `<DateOffset: days=1>` instead of `<DateOffset: kwds={'days': 1}>` ([GH19403](https://github.com/pandas-dev/pandas/issues/19403))  
- `Categorical.fillna` now validates its `value` and `method` keyword arguments. It now raises when both or none are specified, matching the behavior of `Series.fillna()` ([GH19682](https://github.com/pandas-dev/pandas/issues/19682))  
- `pd.to_datetime('today')` now returns a datetime, consistent with `pd.Timestamp('today')`; previously `pd.to_datetime('today')` returned a `.normalized()` datetime ([GH19935](https://github.com/pandas-dev/pandas/issues/19935))  
- `Series.str.replace()` now takes an optional regex keyword which, when set to `False`, uses literal string replacement rather than regex replacement ([GH16808](https://github.com/pandas-dev/pandas/issues/16808))  
- `DatetimeIndex.strftime()` and `PeriodIndex.strftime()` now return an `Index` instead of a numpy array to be consistent with similar accessors ([GH20127](https://github.com/pandas-dev/pandas/issues/20127))  
- Constructing a Series from a list of length 1 no longer broadcasts this list when a longer index is specified ([GH19714](https://github.com/pandas-dev/pandas/issues/19714), [GH20391](https://github.com/pandas-dev/pandas/issues/20391)).  
- `DataFrame.to_dict()` with `orient='index'` no longer casts int columns to float for a DataFrame with only int and float columns ([GH18580](https://github.com/pandas-dev/pandas/issues/18580))  
- A user-defined-function that is passed to `Series.rolling().aggregate()`, `DataFrame.rolling().aggregate()`, or its expanding cousins, will now *always* be passed a `Series`, rather than a `np.array`; `.apply()` only has the `raw` keyword, see [here](#whatsnew-0230-enhancements-window-raw). This is consistent with the signatures of `.aggregate()` across pandas ([GH20584](https://github.com/pandas-dev/pandas/issues/20584))  
- Rolling and Expanding types raise `NotImplementedError` upon iteration ([GH11704](https://github.com/pandas-dev/pandas/issues/11704)).  



<a id='whatsnew-0230-deprecations'></a>

## Deprecations

- `Series.from_array` and `SparseSeries.from_array` are deprecated. Use the normal constructor `Series(..)` and `SparseSeries(..)` instead ([GH18213](https://github.com/pandas-dev/pandas/issues/18213)).  
- `DataFrame.as_matrix` is deprecated. Use `DataFrame.values` instead ([GH18458](https://github.com/pandas-dev/pandas/issues/18458)).  
- `Series.asobject`, `DatetimeIndex.asobject`, `PeriodIndex.asobject` and `TimeDeltaIndex.asobject` have been deprecated. Use `.astype(object)` instead ([GH18572](https://github.com/pandas-dev/pandas/issues/18572))  
- Grouping by a tuple of keys now emits a `FutureWarning` and is deprecated.
  In the future, a tuple passed to `'by'` will always refer to a single key
  that is the actual tuple, instead of treating the tuple as multiple keys. To
  retain the previous behavior, use a list instead of a tuple ([GH18314](https://github.com/pandas-dev/pandas/issues/18314))  
- `Series.valid` is deprecated. Use `Series.dropna()` instead ([GH18800](https://github.com/pandas-dev/pandas/issues/18800)).  
- `read_excel()` has deprecated the `skip_footer` parameter. Use `skipfooter` instead ([GH18836](https://github.com/pandas-dev/pandas/issues/18836))  
- `ExcelFile.parse()` has deprecated `sheetname` in favor of `sheet_name` for consistency with `read_excel()` ([GH20920](https://github.com/pandas-dev/pandas/issues/20920)).  
- The `is_copy` attribute is deprecated and will be removed in a future version ([GH18801](https://github.com/pandas-dev/pandas/issues/18801)).  
- `IntervalIndex.from_intervals` is deprecated in favor of the `IntervalIndex` constructor ([GH19263](https://github.com/pandas-dev/pandas/issues/19263))  
- `DataFrame.from_items` is deprecated. Use `DataFrame.from_dict()` instead, or `DataFrame.from_dict(OrderedDict())` if you wish to preserve the key order ([GH17320](https://github.com/pandas-dev/pandas/issues/17320), [GH17312](https://github.com/pandas-dev/pandas/issues/17312))  
- Indexing a `MultiIndex` or a `FloatIndex` with a list containing some missing keys will now show a [`FutureWarning`](https://docs.python.org/3/library/exceptions.html#FutureWarning), which is consistent with other types of indexes ([GH17758](https://github.com/pandas-dev/pandas/issues/17758)).  
- The `broadcast` parameter of `.apply()` is deprecated in favor of `result_type='broadcast'` ([GH18577](https://github.com/pandas-dev/pandas/issues/18577))  
- The `reduce` parameter of `.apply()` is deprecated in favor of `result_type='reduce'` ([GH18577](https://github.com/pandas-dev/pandas/issues/18577))  
- The `order` parameter of `factorize()` is deprecated and will be removed in a future release ([GH19727](https://github.com/pandas-dev/pandas/issues/19727))  
- `Timestamp.weekday_name`, `DatetimeIndex.weekday_name`, and `Series.dt.weekday_name` are deprecated in favor of `Timestamp.day_name()`, `DatetimeIndex.day_name()`, and `Series.dt.day_name()` ([GH12806](https://github.com/pandas-dev/pandas/issues/12806))  
- `pandas.tseries.plotting.tsplot` is deprecated. Use `Series.plot()` instead ([GH18627](https://github.com/pandas-dev/pandas/issues/18627))  
- `Index.summary()` is deprecated and will be removed in a future version ([GH18217](https://github.com/pandas-dev/pandas/issues/18217))  
- `NDFrame.get_ftype_counts()` is deprecated and will be removed in a future version ([GH18243](https://github.com/pandas-dev/pandas/issues/18243))  
- The `convert_datetime64` parameter in `DataFrame.to_records()` has been deprecated and will be removed in a future version. The NumPy bug motivating this parameter has been resolved. The default value for this parameter has also changed from `True` to `None` ([GH18160](https://github.com/pandas-dev/pandas/issues/18160)).  
- `Series.rolling().apply()`, `DataFrame.rolling().apply()`,
  `Series.expanding().apply()`, and `DataFrame.expanding().apply()` have deprecated passing an `np.array` by default. One will need to pass the new `raw` parameter to be explicit about what is passed ([GH20584](https://github.com/pandas-dev/pandas/issues/20584))  
- The `data`, `base`, `strides`, `flags` and `itemsize` properties
  of the `Series` and `Index` classes have been deprecated and will be
  removed in a future version ([GH20419](https://github.com/pandas-dev/pandas/issues/20419)).  
- `DatetimeIndex.offset` is deprecated. Use `DatetimeIndex.freq` instead ([GH20716](https://github.com/pandas-dev/pandas/issues/20716))  
- Floor division between an integer ndarray and a `Timedelta` is deprecated. Divide by `Timedelta.value` instead ([GH19761](https://github.com/pandas-dev/pandas/issues/19761))  
- Setting `PeriodIndex.freq` (which was not guaranteed to work correctly) is deprecated. Use `PeriodIndex.asfreq()` instead ([GH20678](https://github.com/pandas-dev/pandas/issues/20678))  
- `Index.get_duplicates()` is deprecated and will be removed in a future version ([GH20239](https://github.com/pandas-dev/pandas/issues/20239))  
- The previous default behavior of negative indices in `Categorical.take` is deprecated. In a future version it will change from meaning missing values to meaning positional indices from the right. The future behavior is consistent with `Series.take()` ([GH20664](https://github.com/pandas-dev/pandas/issues/20664)).  
- Passing multiple axes to the `axis` parameter in `DataFrame.dropna()` has been deprecated and will be removed in a future version ([GH20987](https://github.com/pandas-dev/pandas/issues/20987))  



<a id='whatsnew-0230-prior-deprecations'></a>

## Removal of prior version deprecations/changes

- Warnings against the obsolete usage `Categorical(codes, categories)`, which were emitted for instance when the first two arguments to `Categorical()` had different dtypes, and recommended the use of `Categorical.from_codes`, have now been removed ([GH8074](https://github.com/pandas-dev/pandas/issues/8074))  
- The `levels` and `labels` attributes of a `MultiIndex` can no longer be set directly ([GH4039](https://github.com/pandas-dev/pandas/issues/4039)).  
- `pd.tseries.util.pivot_annual` has been removed (deprecated since v0.19). Use `pivot_table` instead ([GH18370](https://github.com/pandas-dev/pandas/issues/18370))  
- `pd.tseries.util.isleapyear` has been removed (deprecated since v0.19). Use `.is_leap_year` property in Datetime-likes instead ([GH18370](https://github.com/pandas-dev/pandas/issues/18370))  
- `pd.ordered_merge` has been removed (deprecated since v0.19). Use `pd.merge_ordered` instead ([GH18459](https://github.com/pandas-dev/pandas/issues/18459))  
- The `SparseList` class has been removed ([GH14007](https://github.com/pandas-dev/pandas/issues/14007))  
- The `pandas.io.wb` and `pandas.io.data` stub modules have been removed ([GH13735](https://github.com/pandas-dev/pandas/issues/13735))  
- `Categorical.from_array` has been removed ([GH13854](https://github.com/pandas-dev/pandas/issues/13854))  
- The `freq` and `how` parameters have been removed from the `rolling`/`expanding`/`ewm` methods of DataFrame
  and Series (deprecated since v0.18). Instead, resample before calling the methods. ([GH18601](https://github.com/pandas-dev/pandas/issues/18601) & [GH18668](https://github.com/pandas-dev/pandas/issues/18668))  
- `DatetimeIndex.to_datetime`, `Timestamp.to_datetime`, `PeriodIndex.to_datetime`, and `Index.to_datetime` have been removed ([GH8254](https://github.com/pandas-dev/pandas/issues/8254), [GH14096](https://github.com/pandas-dev/pandas/issues/14096), [GH14113](https://github.com/pandas-dev/pandas/issues/14113))  
- `read_csv()` has dropped the `skip_footer` parameter ([GH13386](https://github.com/pandas-dev/pandas/issues/13386))  
- `read_csv()` has dropped the `as_recarray` parameter ([GH13373](https://github.com/pandas-dev/pandas/issues/13373))  
- `read_csv()` has dropped the `buffer_lines` parameter ([GH13360](https://github.com/pandas-dev/pandas/issues/13360))  
- `read_csv()` has dropped the `compact_ints` and `use_unsigned` parameters ([GH13323](https://github.com/pandas-dev/pandas/issues/13323))  
- The `Timestamp` class has dropped the `offset` attribute in favor of `freq` ([GH13593](https://github.com/pandas-dev/pandas/issues/13593))  
- The `Series`, `Categorical`, and `Index` classes have dropped the `reshape` method ([GH13012](https://github.com/pandas-dev/pandas/issues/13012))  
- `pandas.tseries.frequencies.get_standard_freq` has been removed in favor of `pandas.tseries.frequencies.to_offset(freq).rule_code` ([GH13874](https://github.com/pandas-dev/pandas/issues/13874))  
- The `freqstr` keyword has been removed from `pandas.tseries.frequencies.to_offset` in favor of `freq` ([GH13874](https://github.com/pandas-dev/pandas/issues/13874))  
- The `Panel4D` and `PanelND` classes have been removed ([GH13776](https://github.com/pandas-dev/pandas/issues/13776))  
- The `Panel` class has dropped the `to_long` and `toLong` methods ([GH19077](https://github.com/pandas-dev/pandas/issues/19077))  
- The options `display.line_with` and `display.height` are removed in favor of `display.width` and `display.max_rows` respectively ([GH4391](https://github.com/pandas-dev/pandas/issues/4391), [GH19107](https://github.com/pandas-dev/pandas/issues/19107))  
- The `labels` attribute of the `Categorical` class has been removed in favor of `Categorical.codes` ([GH7768](https://github.com/pandas-dev/pandas/issues/7768))  
- The `flavor` parameter have been removed from func:to_sql method ([GH13611](https://github.com/pandas-dev/pandas/issues/13611))  
- The modules `pandas.tools.hashing` and `pandas.util.hashing` have been removed ([GH16223](https://github.com/pandas-dev/pandas/issues/16223))  
- The top-level functions `pd.rolling_*`, `pd.expanding_*` and `pd.ewm*` have been removed (Deprecated since v0.18).
  Instead, use the DataFrame/Series methods `rolling`, `expanding` and `ewm` ([GH18723](https://github.com/pandas-dev/pandas/issues/18723))  
- Imports from `pandas.core.common` for functions such as `is_datetime64_dtype` are now removed. These are located in `pandas.api.types`. ([GH13634](https://github.com/pandas-dev/pandas/issues/13634), [GH19769](https://github.com/pandas-dev/pandas/issues/19769))  
- The `infer_dst` keyword in `Series.tz_localize()`, `DatetimeIndex.tz_localize()`
  and `DatetimeIndex` have been removed. `infer_dst=True` is equivalent to
  `ambiguous='infer'`, and `infer_dst=False` to `ambiguous='raise'` ([GH7963](https://github.com/pandas-dev/pandas/issues/7963)).  
- When `.resample()` was changed from an eager to a lazy operation, like `.groupby()` in v0.18.0, we put in place compatibility (with a `FutureWarning`),
  so operations would continue to work. This is now fully removed, so a `Resampler` will no longer forward compat operations ([GH20554](https://github.com/pandas-dev/pandas/issues/20554))  
- Remove long deprecated `axis=None` parameter from `.replace()` ([GH20271](https://github.com/pandas-dev/pandas/issues/20271))  



<a id='whatsnew-0230-performance'></a>

## Performance improvements

- Indexers on `Series` or `DataFrame` no longer create a reference cycle ([GH17956](https://github.com/pandas-dev/pandas/issues/17956))  
- Added a keyword argument, `cache`, to `to_datetime()` that improved the performance of converting duplicate datetime arguments ([GH11665](https://github.com/pandas-dev/pandas/issues/11665))  
- `DateOffset` arithmetic performance is improved ([GH18218](https://github.com/pandas-dev/pandas/issues/18218))  
- Converting a `Series` of `Timedelta` objects to days, seconds, etc… sped up through vectorization of underlying methods ([GH18092](https://github.com/pandas-dev/pandas/issues/18092))  
- Improved performance of `.map()` with a `Series/dict` input ([GH15081](https://github.com/pandas-dev/pandas/issues/15081))  
- The overridden `Timedelta` properties of days, seconds and microseconds have been removed, leveraging their built-in Python versions instead ([GH18242](https://github.com/pandas-dev/pandas/issues/18242))  
- `Series` construction will reduce the number of copies made of the input data in certain cases ([GH17449](https://github.com/pandas-dev/pandas/issues/17449))  
- Improved performance of `Series.dt.date()` and `DatetimeIndex.date()` ([GH18058](https://github.com/pandas-dev/pandas/issues/18058))  
- Improved performance of `Series.dt.time()` and `DatetimeIndex.time()` ([GH18461](https://github.com/pandas-dev/pandas/issues/18461))  
- Improved performance of `IntervalIndex.symmetric_difference()` ([GH18475](https://github.com/pandas-dev/pandas/issues/18475))  
- Improved performance of `DatetimeIndex` and `Series` arithmetic operations with Business-Month and Business-Quarter frequencies ([GH18489](https://github.com/pandas-dev/pandas/issues/18489))  
- `Series()` / `DataFrame()` tab completion limits to 100 values, for better performance. ([GH18587](https://github.com/pandas-dev/pandas/issues/18587))  
- Improved performance of `DataFrame.median()` with `axis=1` when bottleneck is not installed ([GH16468](https://github.com/pandas-dev/pandas/issues/16468))  
- Improved performance of `MultiIndex.get_loc()` for large indexes, at the cost of a reduction in performance for small ones ([GH18519](https://github.com/pandas-dev/pandas/issues/18519))  
- Improved performance of `MultiIndex.remove_unused_levels()` when there are no unused levels, at the cost of a reduction in performance when there are ([GH19289](https://github.com/pandas-dev/pandas/issues/19289))  
- Improved performance of `Index.get_loc()` for non-unique indexes ([GH19478](https://github.com/pandas-dev/pandas/issues/19478))  
- Improved performance of pairwise `.rolling()` and `.expanding()` with `.cov()` and `.corr()` operations ([GH17917](https://github.com/pandas-dev/pandas/issues/17917))  
- Improved performance of `pandas.core.groupby.GroupBy.rank()` ([GH15779](https://github.com/pandas-dev/pandas/issues/15779))  
- Improved performance of variable `.rolling()` on `.min()` and `.max()` ([GH19521](https://github.com/pandas-dev/pandas/issues/19521))  
- Improved performance of `pandas.core.groupby.GroupBy.ffill()` and `pandas.core.groupby.GroupBy.bfill()` ([GH11296](https://github.com/pandas-dev/pandas/issues/11296))  
- Improved performance of `pandas.core.groupby.GroupBy.any()` and `pandas.core.groupby.GroupBy.all()` ([GH15435](https://github.com/pandas-dev/pandas/issues/15435))  
- Improved performance of `pandas.core.groupby.GroupBy.pct_change()` ([GH19165](https://github.com/pandas-dev/pandas/issues/19165))  
- Improved performance of `Series.isin()` in the case of categorical dtypes ([GH20003](https://github.com/pandas-dev/pandas/issues/20003))  
- Improved performance of `getattr(Series, attr)` when the Series has certain index types. This manifested in slow printing of large Series with a `DatetimeIndex` ([GH19764](https://github.com/pandas-dev/pandas/issues/19764))  
- Fixed a performance regression for `GroupBy.nth()` and `GroupBy.last()` with some object columns ([GH19283](https://github.com/pandas-dev/pandas/issues/19283))  
- Improved performance of `pandas.core.arrays.Categorical.from_codes()` ([GH18501](https://github.com/pandas-dev/pandas/issues/18501))  



<a id='whatsnew-0230-docs'></a>

## Documentation changes

Thanks to all of the contributors who participated in the Pandas Documentation
Sprint, which took place on March 10th. We had about 500 participants from over
30 locations across the world. You should notice that many of the
[API docstrings](https://pandas-gbq.readthedocs.io/en/latest/api.html#api) have greatly improved.

There were too many simultaneous contributions to include a release note for each
improvement, but this [GitHub search](https://github.com/pandas-dev/pandas/pulls?utf8=%E2%9C%93&q=is%3Apr+label%3ADocs+created%3A2018-03-10..2018-03-15+) should give you an idea of how many docstrings
were improved.

Special thanks to [Marc Garcia](https://github.com/datapythonista) for organizing the sprint. For more information,
read the [NumFOCUS blogpost](https://www.numfocus.org/blog/worldwide-pandas-sprint/) recapping the sprint.

- Changed spelling of “numpy” to “NumPy”, and “python” to “Python”. ([GH19017](https://github.com/pandas-dev/pandas/issues/19017))  
- Consistency when introducing code samples, using either colon or period.
  Rewrote some sentences for greater clarity, added more dynamic references
  to functions, methods and classes.
  ([GH18941](https://github.com/pandas-dev/pandas/issues/18941), [GH18948](https://github.com/pandas-dev/pandas/issues/18948), [GH18973](https://github.com/pandas-dev/pandas/issues/18973), [GH19017](https://github.com/pandas-dev/pandas/issues/19017))  
- Added a reference to `DataFrame.assign()` in the concatenate section of the merging documentation ([GH18665](https://github.com/pandas-dev/pandas/issues/18665))  



<a id='whatsnew-0230-bug-fixes'></a>

## Bug fixes

### Categorical

A class of bugs were introduced in pandas 0.21 with `CategoricalDtype` that
affects the correctness of operations like `merge`, `concat`, and
indexing when comparing multiple unordered `Categorical` arrays that have
the same categories, but in a different order. We highly recommend upgrading
or manually aligning your categories before doing these operations.

- Bug in `Categorical.equals` returning the wrong result when comparing two
  unordered `Categorical` arrays with the same categories, but in a different
  order ([GH16603](https://github.com/pandas-dev/pandas/issues/16603))  
- Bug in `pandas.api.types.union_categoricals()` returning the wrong result
  when for unordered categoricals with the categories in a different order.
  This affected `pandas.concat()` with Categorical data ([GH19096](https://github.com/pandas-dev/pandas/issues/19096)).  
- Bug in `pandas.merge()` returning the wrong result when joining on an
  unordered `Categorical` that had the same categories but in a different
  order ([GH19551](https://github.com/pandas-dev/pandas/issues/19551))  
- Bug in `CategoricalIndex.get_indexer()` returning the wrong result when
  `target` was an unordered `Categorical` that had the same categories as
  `self` but in a different order ([GH19551](https://github.com/pandas-dev/pandas/issues/19551))  
- Bug in `Index.astype()` with a categorical dtype where the resultant index is not converted to a `CategoricalIndex` for all types of index ([GH18630](https://github.com/pandas-dev/pandas/issues/18630))  
- Bug in `Series.astype()` and `Categorical.astype()` where an existing categorical data does not get updated ([GH10696](https://github.com/pandas-dev/pandas/issues/10696), [GH18593](https://github.com/pandas-dev/pandas/issues/18593))  
- Bug in `Series.str.split()` with `expand=True` incorrectly raising an IndexError on empty strings ([GH20002](https://github.com/pandas-dev/pandas/issues/20002)).  
- Bug in `Index` constructor with `dtype=CategoricalDtype(...)` where `categories` and `ordered` are not maintained ([GH19032](https://github.com/pandas-dev/pandas/issues/19032))  
- Bug in `Series` constructor with scalar and `dtype=CategoricalDtype(...)` where `categories` and `ordered` are not maintained ([GH19565](https://github.com/pandas-dev/pandas/issues/19565))  
- Bug in `Categorical.__iter__` not converting to Python types ([GH19909](https://github.com/pandas-dev/pandas/issues/19909))  
- Bug in `pandas.factorize()` returning the unique codes for the `uniques`. This now returns a `Categorical` with the same dtype as the input ([GH19721](https://github.com/pandas-dev/pandas/issues/19721))  
- Bug in `pandas.factorize()` including an item for missing values in the `uniques` return value ([GH19721](https://github.com/pandas-dev/pandas/issues/19721))  
- Bug in `Series.take()` with categorical data interpreting `-1` in indices as missing value markers, rather than the last element of the Series ([GH20664](https://github.com/pandas-dev/pandas/issues/20664))  

### Datetimelike

- Bug in `Series.__sub__()` subtracting a non-nanosecond `np.datetime64` object from a `Series` gave incorrect results ([GH7996](https://github.com/pandas-dev/pandas/issues/7996))  
- Bug in `DatetimeIndex`, `TimedeltaIndex` addition and subtraction of zero-dimensional integer arrays gave incorrect results ([GH19012](https://github.com/pandas-dev/pandas/issues/19012))  
- Bug in `DatetimeIndex` and `TimedeltaIndex` where adding or subtracting an array-like of `DateOffset` objects either raised (`np.array`, `pd.Index`) or broadcast incorrectly (`pd.Series`) ([GH18849](https://github.com/pandas-dev/pandas/issues/18849))  
- Bug in `Series.__add__()` adding Series with dtype `timedelta64[ns]` to a timezone-aware `DatetimeIndex` incorrectly dropped timezone information ([GH13905](https://github.com/pandas-dev/pandas/issues/13905))  
- Adding a `Period` object to a `datetime` or `Timestamp` object will now correctly raise a `TypeError` ([GH17983](https://github.com/pandas-dev/pandas/issues/17983))  
- Bug in `Timestamp` where comparison with an array of `Timestamp` objects would result in a `RecursionError` ([GH15183](https://github.com/pandas-dev/pandas/issues/15183))  
- Bug in `Series` floor-division where operating on a scalar `timedelta` raises an exception ([GH18846](https://github.com/pandas-dev/pandas/issues/18846))  
- Bug in `DatetimeIndex` where the repr was not showing high-precision time values at the end of a day (e.g., 23:59:59.999999999) ([GH19030](https://github.com/pandas-dev/pandas/issues/19030))  
- Bug in `.astype()` to non-ns timedelta units would hold the incorrect dtype ([GH19176](https://github.com/pandas-dev/pandas/issues/19176), [GH19223](https://github.com/pandas-dev/pandas/issues/19223), [GH12425](https://github.com/pandas-dev/pandas/issues/12425))  
- Bug in subtracting `Series` from `NaT` incorrectly returning `NaT` ([GH19158](https://github.com/pandas-dev/pandas/issues/19158))  
- Bug in `Series.truncate()` which raises `TypeError` with a monotonic `PeriodIndex` ([GH17717](https://github.com/pandas-dev/pandas/issues/17717))  
- Bug in `pct_change()` using `periods` and `freq` returned different length outputs ([GH7292](https://github.com/pandas-dev/pandas/issues/7292))  
- Bug in comparison of `DatetimeIndex` against `None` or `datetime.date` objects raising `TypeError` for `==` and `!=` comparisons instead of all-`False` and all-`True`, respectively ([GH19301](https://github.com/pandas-dev/pandas/issues/19301))  
- Bug in `Timestamp` and `to_datetime()` where a string representing a barely out-of-bounds timestamp would be incorrectly rounded down instead of raising `OutOfBoundsDatetime` ([GH19382](https://github.com/pandas-dev/pandas/issues/19382))  
- Bug in `Timestamp.floor()` `DatetimeIndex.floor()` where time stamps far in the future and past were not rounded correctly ([GH19206](https://github.com/pandas-dev/pandas/issues/19206))  
- Bug in `to_datetime()` where passing an out-of-bounds datetime with `errors='coerce'` and `utc=True` would raise `OutOfBoundsDatetime` instead of parsing to `NaT` ([GH19612](https://github.com/pandas-dev/pandas/issues/19612))  
- Bug in `DatetimeIndex` and `TimedeltaIndex` addition and subtraction where name of the returned object was not always set consistently. ([GH19744](https://github.com/pandas-dev/pandas/issues/19744))  
- Bug in `DatetimeIndex` and `TimedeltaIndex` addition and subtraction where operations with numpy arrays raised `TypeError` ([GH19847](https://github.com/pandas-dev/pandas/issues/19847))  
- Bug in `DatetimeIndex` and `TimedeltaIndex` where setting the `freq` attribute was not fully supported ([GH20678](https://github.com/pandas-dev/pandas/issues/20678))  

### Timedelta

- Bug in `Timedelta.__mul__()` where multiplying by `NaT` returned `NaT` instead of raising a `TypeError` ([GH19819](https://github.com/pandas-dev/pandas/issues/19819))  
- Bug in `Series` with `dtype='timedelta64[ns]'` where addition or subtraction of `TimedeltaIndex` had results cast to `dtype='int64'` ([GH17250](https://github.com/pandas-dev/pandas/issues/17250))  
- Bug in `Series` with `dtype='timedelta64[ns]'` where addition or subtraction of `TimedeltaIndex` could return a `Series` with an incorrect name ([GH19043](https://github.com/pandas-dev/pandas/issues/19043))  
- Bug in `Timedelta.__floordiv__()` and `Timedelta.__rfloordiv__()` dividing by many incompatible numpy objects was incorrectly allowed ([GH18846](https://github.com/pandas-dev/pandas/issues/18846))  
- Bug where dividing a scalar timedelta-like object with `TimedeltaIndex` performed the reciprocal operation ([GH19125](https://github.com/pandas-dev/pandas/issues/19125))  
- Bug in `TimedeltaIndex` where division by a `Series` would return a `TimedeltaIndex` instead of a `Series` ([GH19042](https://github.com/pandas-dev/pandas/issues/19042))  
- Bug in `Timedelta.__add__()`, `Timedelta.__sub__()` where adding or subtracting a `np.timedelta64` object would return another `np.timedelta64` instead of a `Timedelta` ([GH19738](https://github.com/pandas-dev/pandas/issues/19738))  
- Bug in `Timedelta.__floordiv__()`, `Timedelta.__rfloordiv__()` where operating with a `Tick` object would raise a `TypeError` instead of returning a numeric value ([GH19738](https://github.com/pandas-dev/pandas/issues/19738))  
- Bug in `Period.asfreq()` where periods near `datetime(1, 1, 1)` could be converted incorrectly ([GH19643](https://github.com/pandas-dev/pandas/issues/19643), [GH19834](https://github.com/pandas-dev/pandas/issues/19834))  
- Bug in `Timedelta.total_seconds()` causing precision errors, for example `Timedelta('30S').total_seconds()==30.000000000000004` ([GH19458](https://github.com/pandas-dev/pandas/issues/19458))  
- Bug in `Timedelta.__rmod__()` where operating with a `numpy.timedelta64` returned a `timedelta64` object instead of a `Timedelta` ([GH19820](https://github.com/pandas-dev/pandas/issues/19820))  
- Multiplication of `TimedeltaIndex` by `TimedeltaIndex` will now raise `TypeError` instead of raising `ValueError` in cases of length mis-match ([GH19333](https://github.com/pandas-dev/pandas/issues/19333))  
- Bug in indexing a `TimedeltaIndex` with a `np.timedelta64` object which was raising a `TypeError` ([GH20393](https://github.com/pandas-dev/pandas/issues/20393))  

### Timezones

- Bug in creating a `Series` from an array that contains both tz-naive and tz-aware values will result in a `Series` whose dtype is tz-aware instead of object ([GH16406](https://github.com/pandas-dev/pandas/issues/16406))  
- Bug in comparison of timezone-aware `DatetimeIndex` against `NaT` incorrectly raising `TypeError` ([GH19276](https://github.com/pandas-dev/pandas/issues/19276))  
- Bug in `DatetimeIndex.astype()` when converting between timezone aware dtypes, and converting from timezone aware to naive ([GH18951](https://github.com/pandas-dev/pandas/issues/18951))  
- Bug in comparing `DatetimeIndex`, which failed to raise `TypeError` when attempting to compare timezone-aware and timezone-naive datetimelike objects ([GH18162](https://github.com/pandas-dev/pandas/issues/18162))  
- Bug in localization of a naive, datetime string in a `Series` constructor with a `datetime64[ns, tz]` dtype ([GH174151](https://github.com/pandas-dev/pandas/issues/174151))  
- `Timestamp.replace()` will now handle Daylight Savings transitions gracefully ([GH18319](https://github.com/pandas-dev/pandas/issues/18319))  
- Bug in tz-aware `DatetimeIndex` where addition/subtraction with a `TimedeltaIndex` or array with `dtype='timedelta64[ns]'` was incorrect ([GH17558](https://github.com/pandas-dev/pandas/issues/17558))  
- Bug in `DatetimeIndex.insert()` where inserting `NaT` into a timezone-aware index incorrectly raised ([GH16357](https://github.com/pandas-dev/pandas/issues/16357))  
- Bug in `DataFrame` constructor, where tz-aware Datetimeindex and a given column name will result in an empty `DataFrame` ([GH19157](https://github.com/pandas-dev/pandas/issues/19157))  
- Bug in `Timestamp.tz_localize()` where localizing a timestamp near the minimum or maximum valid values could overflow and return a timestamp with an incorrect nanosecond value ([GH12677](https://github.com/pandas-dev/pandas/issues/12677))  
- Bug when iterating over `DatetimeIndex` that was localized with fixed timezone offset that rounded nanosecond precision to microseconds ([GH19603](https://github.com/pandas-dev/pandas/issues/19603))  
- Bug in `DataFrame.diff()` that raised an `IndexError` with tz-aware values ([GH18578](https://github.com/pandas-dev/pandas/issues/18578))  
- Bug in `melt()` that converted tz-aware dtypes to tz-naive ([GH15785](https://github.com/pandas-dev/pandas/issues/15785))  
- Bug in `Dataframe.count()` that raised an `ValueError`, if `Dataframe.dropna()` was called for a single column with timezone-aware values. ([GH13407](https://github.com/pandas-dev/pandas/issues/13407))  

### Offsets

- Bug in `WeekOfMonth` and `Week` where addition and subtraction did not roll correctly ([GH18510](https://github.com/pandas-dev/pandas/issues/18510), [GH18672](https://github.com/pandas-dev/pandas/issues/18672), [GH18864](https://github.com/pandas-dev/pandas/issues/18864))  
- Bug in `WeekOfMonth` and `LastWeekOfMonth` where default keyword arguments for constructor raised `ValueError` ([GH19142](https://github.com/pandas-dev/pandas/issues/19142))  
- Bug in `FY5253Quarter`, `LastWeekOfMonth` where rollback and rollforward behavior was inconsistent with addition and subtraction behavior ([GH18854](https://github.com/pandas-dev/pandas/issues/18854))  
- Bug in `FY5253` where `datetime` addition and subtraction incremented incorrectly for dates on the year-end but not normalized to midnight ([GH18854](https://github.com/pandas-dev/pandas/issues/18854))  
- Bug in `FY5253` where date offsets could incorrectly raise an `AssertionError` in arithmetic operations ([GH14774](https://github.com/pandas-dev/pandas/issues/14774))  

### Numeric

- Bug in `Series` constructor with an int or float list where specifying `dtype=str`, `dtype='str'` or `dtype='U'` failed to convert the data elements to strings ([GH16605](https://github.com/pandas-dev/pandas/issues/16605))  
- Bug in `Index` multiplication and division methods where operating with a `Series` would return an `Index` object instead of a `Series` object ([GH19042](https://github.com/pandas-dev/pandas/issues/19042))  
- Bug in the `DataFrame` constructor in which data containing very large positive or very large negative numbers was causing `OverflowError` ([GH18584](https://github.com/pandas-dev/pandas/issues/18584))  
- Bug in `Index` constructor with `dtype='uint64'` where int-like floats were not coerced to `UInt64Index` ([GH18400](https://github.com/pandas-dev/pandas/issues/18400))  
- Bug in `DataFrame` flex arithmetic (e.g. `df.add(other, fill_value=foo)`) with a `fill_value` other than `None` failed to raise `NotImplementedError` in corner cases where either the frame or `other` has length zero ([GH19522](https://github.com/pandas-dev/pandas/issues/19522))  
- Multiplication and division of numeric-dtyped `Index` objects with timedelta-like scalars returns `TimedeltaIndex` instead of raising `TypeError` ([GH19333](https://github.com/pandas-dev/pandas/issues/19333))  
- Bug where `NaN` was returned instead of 0 by `Series.pct_change()` and `DataFrame.pct_change()` when `fill_method` is not `None` ([GH19873](https://github.com/pandas-dev/pandas/issues/19873))  

### Strings

- Bug in `Series.str.get()` with a dictionary in the values and the index not in the keys, raising KeyError ([GH20671](https://github.com/pandas-dev/pandas/issues/20671))  

### Indexing

- Bug in `Index` construction from list of mixed type tuples ([GH18505](https://github.com/pandas-dev/pandas/issues/18505))  
- Bug in `Index.drop()` when passing a list of both tuples and non-tuples ([GH18304](https://github.com/pandas-dev/pandas/issues/18304))  
- Bug in `DataFrame.drop()`, `Panel.drop()`, `Series.drop()`, `Index.drop()` where no `KeyError` is raised when dropping a non-existent element from an axis that contains duplicates ([GH19186](https://github.com/pandas-dev/pandas/issues/19186))  
- Bug in indexing a datetimelike `Index` that raised `ValueError` instead of `IndexError` ([GH18386](https://github.com/pandas-dev/pandas/issues/18386)).  
- `Index.to_series()` now accepts `index` and `name` kwargs ([GH18699](https://github.com/pandas-dev/pandas/issues/18699))  
- `DatetimeIndex.to_series()` now accepts `index` and `name` kwargs ([GH18699](https://github.com/pandas-dev/pandas/issues/18699))  
- Bug in indexing non-scalar value from `Series` having non-unique `Index` will return value flattened ([GH17610](https://github.com/pandas-dev/pandas/issues/17610))  
- Bug in indexing with iterator containing only missing keys, which raised no error ([GH20748](https://github.com/pandas-dev/pandas/issues/20748))  
- Fixed inconsistency in `.ix` between list and scalar keys when the index has integer dtype and does not include the desired keys ([GH20753](https://github.com/pandas-dev/pandas/issues/20753))  
- Bug in `__setitem__` when indexing a `DataFrame` with a 2-d boolean ndarray ([GH18582](https://github.com/pandas-dev/pandas/issues/18582))  
- Bug in `str.extractall` when there were no matches empty `Index` was returned instead of appropriate `MultiIndex` ([GH19034](https://github.com/pandas-dev/pandas/issues/19034))  
- Bug in `IntervalIndex` where empty and purely NA data was constructed inconsistently depending on the construction method ([GH18421](https://github.com/pandas-dev/pandas/issues/18421))  
- Bug in `IntervalIndex.symmetric_difference()` where the symmetric difference with a non-`IntervalIndex` did not raise ([GH18475](https://github.com/pandas-dev/pandas/issues/18475))  
- Bug in `IntervalIndex` where set operations that returned an empty `IntervalIndex` had the wrong dtype ([GH19101](https://github.com/pandas-dev/pandas/issues/19101))  
- Bug in `DataFrame.drop_duplicates()` where no `KeyError` is raised when passing in columns that don’t exist on the `DataFrame` ([GH19726](https://github.com/pandas-dev/pandas/issues/19726))  
- Bug in `Index` subclasses constructors that ignore unexpected keyword arguments ([GH19348](https://github.com/pandas-dev/pandas/issues/19348))  
- Bug in `Index.difference()` when taking difference of an `Index` with itself ([GH20040](https://github.com/pandas-dev/pandas/issues/20040))  
- Bug in `DataFrame.first_valid_index()` and `DataFrame.last_valid_index()` in presence of entire rows of NaNs in the middle of values ([GH20499](https://github.com/pandas-dev/pandas/issues/20499)).  
- Bug in `IntervalIndex` where some indexing operations were not supported for overlapping or non-monotonic `uint64` data ([GH20636](https://github.com/pandas-dev/pandas/issues/20636))  
- Bug in `Series.is_unique` where extraneous output in stderr is shown if Series contains objects with `__ne__` defined ([GH20661](https://github.com/pandas-dev/pandas/issues/20661))  
- Bug in `.loc` assignment with a single-element list-like incorrectly assigns as a list ([GH19474](https://github.com/pandas-dev/pandas/issues/19474))  
- Bug in partial string indexing on a `Series/DataFrame` with a monotonic decreasing `DatetimeIndex` ([GH19362](https://github.com/pandas-dev/pandas/issues/19362))  
- Bug in performing in-place operations on a `DataFrame` with a duplicate `Index` ([GH17105](https://github.com/pandas-dev/pandas/issues/17105))  
- Bug in `IntervalIndex.get_loc()` and `IntervalIndex.get_indexer()` when used with an `IntervalIndex` containing a single interval ([GH17284](https://github.com/pandas-dev/pandas/issues/17284), [GH20921](https://github.com/pandas-dev/pandas/issues/20921))  
- Bug in `.loc` with a `uint64` indexer ([GH20722](https://github.com/pandas-dev/pandas/issues/20722))  

### MultiIndex

- Bug in `MultiIndex.__contains__()` where non-tuple keys would return `True` even if they had been dropped ([GH19027](https://github.com/pandas-dev/pandas/issues/19027))  
- Bug in `MultiIndex.set_labels()` which would cause casting (and potentially clipping) of the new labels if the `level` argument is not 0 or a list like [0, 1, … ]  ([GH19057](https://github.com/pandas-dev/pandas/issues/19057))  
- Bug in `MultiIndex.get_level_values()` which would return an invalid index on level of ints with missing values ([GH17924](https://github.com/pandas-dev/pandas/issues/17924))  
- Bug in `MultiIndex.unique()` when called on empty `MultiIndex` ([GH20568](https://github.com/pandas-dev/pandas/issues/20568))  
- Bug in `MultiIndex.unique()` which would not preserve level names ([GH20570](https://github.com/pandas-dev/pandas/issues/20570))  
- Bug in `MultiIndex.remove_unused_levels()` which would fill nan values ([GH18417](https://github.com/pandas-dev/pandas/issues/18417))  
- Bug in `MultiIndex.from_tuples()` which would fail to take zipped tuples in python3 ([GH18434](https://github.com/pandas-dev/pandas/issues/18434))  
- Bug in `MultiIndex.get_loc()` which would fail to automatically cast values between float and int ([GH18818](https://github.com/pandas-dev/pandas/issues/18818), [GH15994](https://github.com/pandas-dev/pandas/issues/15994))  
- Bug in `MultiIndex.get_loc()` which would cast boolean to integer labels ([GH19086](https://github.com/pandas-dev/pandas/issues/19086))  
- Bug in `MultiIndex.get_loc()` which would fail to locate keys containing `NaN` ([GH18485](https://github.com/pandas-dev/pandas/issues/18485))  
- Bug in `MultiIndex.get_loc()` in large `MultiIndex`, would fail when levels had different dtypes ([GH18520](https://github.com/pandas-dev/pandas/issues/18520))  
- Bug in indexing where nested indexers having only numpy arrays are handled incorrectly ([GH19686](https://github.com/pandas-dev/pandas/issues/19686))  

### I/O

- `read_html()` now rewinds seekable IO objects after parse failure, before attempting to parse with a new parser. If a parser errors and the object is non-seekable, an informative error is raised suggesting the use of a different parser ([GH17975](https://github.com/pandas-dev/pandas/issues/17975))  
- `DataFrame.to_html()` now has an option to add an id to the leading <table> tag ([GH8496](https://github.com/pandas-dev/pandas/issues/8496))  
- Bug in `read_msgpack()` with a non existent file is passed in Python 2 ([GH15296](https://github.com/pandas-dev/pandas/issues/15296))  
- Bug in `read_csv()` where a `MultiIndex` with duplicate columns was not being mangled appropriately ([GH18062](https://github.com/pandas-dev/pandas/issues/18062))  
- Bug in `read_csv()` where missing values were not being handled properly when `keep_default_na=False` with dictionary `na_values` ([GH19227](https://github.com/pandas-dev/pandas/issues/19227))  
- Bug in `read_csv()` causing heap corruption on 32-bit, big-endian architectures ([GH20785](https://github.com/pandas-dev/pandas/issues/20785))  
- Bug in `read_sas()` where a file with 0 variables gave an `AttributeError` incorrectly. Now it gives an `EmptyDataError` ([GH18184](https://github.com/pandas-dev/pandas/issues/18184))  
- Bug in `DataFrame.to_latex()` where pairs of braces meant to serve as invisible placeholders were escaped ([GH18667](https://github.com/pandas-dev/pandas/issues/18667))  
- Bug in `DataFrame.to_latex()` where a `NaN` in a `MultiIndex` would cause an `IndexError` or incorrect output ([GH14249](https://github.com/pandas-dev/pandas/issues/14249))  
- Bug in `DataFrame.to_latex()` where a non-string index-level name would result in an `AttributeError` ([GH19981](https://github.com/pandas-dev/pandas/issues/19981))  
- Bug in `DataFrame.to_latex()` where the combination of an index name and the index_names=False option would result in incorrect output ([GH18326](https://github.com/pandas-dev/pandas/issues/18326))  
- Bug in `DataFrame.to_latex()` where a `MultiIndex` with an empty string as its name would result in incorrect output ([GH18669](https://github.com/pandas-dev/pandas/issues/18669))  
- Bug in `DataFrame.to_latex()` where missing space characters caused wrong escaping and produced non-valid latex in some cases ([GH20859](https://github.com/pandas-dev/pandas/issues/20859))  
- Bug in `read_json()` where large numeric values were causing an `OverflowError` ([GH18842](https://github.com/pandas-dev/pandas/issues/18842))  
- Bug in `DataFrame.to_parquet()` where an exception was raised if the write destination is S3 ([GH19134](https://github.com/pandas-dev/pandas/issues/19134))  
- `Interval` now supported in `DataFrame.to_excel()` for all Excel file types ([GH19242](https://github.com/pandas-dev/pandas/issues/19242))  
- `Timedelta` now supported in `DataFrame.to_excel()` for all Excel file types ([GH19242](https://github.com/pandas-dev/pandas/issues/19242), [GH9155](https://github.com/pandas-dev/pandas/issues/9155), [GH19900](https://github.com/pandas-dev/pandas/issues/19900))  
- Bug in `pandas.io.stata.StataReader.value_labels()` raising an `AttributeError` when called on very old files. Now returns an empty dict ([GH19417](https://github.com/pandas-dev/pandas/issues/19417))  
- Bug in `read_pickle()` when unpickling objects with `TimedeltaIndex` or `Float64Index` created with pandas prior to version 0.20 ([GH19939](https://github.com/pandas-dev/pandas/issues/19939))  
- Bug in `pandas.io.json.json_normalize()` where sub-records are not properly normalized if any sub-records values are NoneType ([GH20030](https://github.com/pandas-dev/pandas/issues/20030))  
- Bug in `usecols` parameter in `read_csv()` where error is not raised correctly when passing a string. ([GH20529](https://github.com/pandas-dev/pandas/issues/20529))  
- Bug in `HDFStore.keys()` when reading a file with a soft link causes exception ([GH20523](https://github.com/pandas-dev/pandas/issues/20523))  
- Bug in `HDFStore.select_column()` where a key which is not a valid store raised an `AttributeError` instead of a `KeyError` ([GH17912](https://github.com/pandas-dev/pandas/issues/17912))  

### Plotting

- Better error message when attempting to plot but matplotlib is not installed ([GH19810](https://github.com/pandas-dev/pandas/issues/19810)).  
- `DataFrame.plot()` now raises a `ValueError` when the `x` or `y` argument is improperly formed ([GH18671](https://github.com/pandas-dev/pandas/issues/18671))  
- Bug in `DataFrame.plot()` when `x` and `y` arguments given as positions caused incorrect referenced columns for line, bar and area plots ([GH20056](https://github.com/pandas-dev/pandas/issues/20056))  
- Bug in formatting tick labels with `datetime.time()` and fractional seconds ([GH18478](https://github.com/pandas-dev/pandas/issues/18478)).  
- `Series.plot.kde()` has exposed the args `ind` and `bw_method` in the docstring ([GH18461](https://github.com/pandas-dev/pandas/issues/18461)). The argument `ind` may now also be an integer (number of sample points).  
- `DataFrame.plot()` now supports multiple columns to the `y` argument ([GH19699](https://github.com/pandas-dev/pandas/issues/19699))  

### Groupby/resample/rolling

- Bug when grouping by a single column and aggregating with a class like `list` or `tuple` ([GH18079](https://github.com/pandas-dev/pandas/issues/18079))  
- Fixed regression in `DataFrame.groupby()` which would not emit an error when called with a tuple key not in the index ([GH18798](https://github.com/pandas-dev/pandas/issues/18798))  
- Bug in `DataFrame.resample()` which silently ignored unsupported (or mistyped) options for `label`, `closed` and `convention` ([GH19303](https://github.com/pandas-dev/pandas/issues/19303))  
- Bug in `DataFrame.groupby()` where tuples were interpreted as lists of keys rather than as keys ([GH17979](https://github.com/pandas-dev/pandas/issues/17979), [GH18249](https://github.com/pandas-dev/pandas/issues/18249))  
- Bug in `DataFrame.groupby()` where aggregation by `first`/`last`/`min`/`max` was causing timestamps to lose precision ([GH19526](https://github.com/pandas-dev/pandas/issues/19526))  
- Bug in `DataFrame.transform()` where particular aggregation functions were being incorrectly cast to match the dtype(s) of the grouped data ([GH19200](https://github.com/pandas-dev/pandas/issues/19200))  
- Bug in `DataFrame.groupby()` passing the on= kwarg, and subsequently using `.apply()` ([GH17813](https://github.com/pandas-dev/pandas/issues/17813))  
- Bug in `DataFrame.resample().aggregate` not raising a `KeyError` when aggregating a non-existent column ([GH16766](https://github.com/pandas-dev/pandas/issues/16766), [GH19566](https://github.com/pandas-dev/pandas/issues/19566))  
- Bug in `DataFrameGroupBy.cumsum()` and `DataFrameGroupBy.cumprod()` when `skipna` was passed ([GH19806](https://github.com/pandas-dev/pandas/issues/19806))  
- Bug in `DataFrame.resample()` that dropped timezone information ([GH13238](https://github.com/pandas-dev/pandas/issues/13238))  
- Bug in `DataFrame.groupby()` where transformations using `np.all` and `np.any` were raising a `ValueError` ([GH20653](https://github.com/pandas-dev/pandas/issues/20653))  
- Bug in `DataFrame.resample()` where `ffill`, `bfill`, `pad`, `backfill`, `fillna`, `interpolate`, and `asfreq` were ignoring `loffset`. ([GH20744](https://github.com/pandas-dev/pandas/issues/20744))  
- Bug in `DataFrame.groupby()` when applying a function that has mixed data types and the user supplied function can fail on the grouping column ([GH20949](https://github.com/pandas-dev/pandas/issues/20949))  
- Bug in `DataFrameGroupBy.rolling().apply()` where operations performed against the associated `DataFrameGroupBy` object could impact the inclusion of the grouped item(s) in the result ([GH14013](https://github.com/pandas-dev/pandas/issues/14013))  

### Sparse

- Bug in which creating a `SparseDataFrame` from a dense `Series` or an unsupported type raised an uncontrolled exception ([GH19374](https://github.com/pandas-dev/pandas/issues/19374))  
- Bug in `SparseDataFrame.to_csv` causing exception ([GH19384](https://github.com/pandas-dev/pandas/issues/19384))  
- Bug in `SparseSeries.memory_usage` which caused segfault by accessing non sparse elements ([GH19368](https://github.com/pandas-dev/pandas/issues/19368))  
- Bug in constructing a `SparseArray`: if `data` is a scalar and `index` is defined it will coerce to `float64` regardless of scalar’s dtype. ([GH19163](https://github.com/pandas-dev/pandas/issues/19163))  

### Reshaping

- Bug in `DataFrame.merge()` where referencing a `CategoricalIndex` by name, where the `by` kwarg would `KeyError` ([GH20777](https://github.com/pandas-dev/pandas/issues/20777))  
- Bug in `DataFrame.stack()` which fails trying to sort mixed type levels under Python 3 ([GH18310](https://github.com/pandas-dev/pandas/issues/18310))  
- Bug in `DataFrame.unstack()` which casts int to float if `columns` is a `MultiIndex` with unused levels ([GH17845](https://github.com/pandas-dev/pandas/issues/17845))  
- Bug in `DataFrame.unstack()` which raises an error if `index` is a `MultiIndex` with unused labels on the unstacked level ([GH18562](https://github.com/pandas-dev/pandas/issues/18562))  
- Fixed construction of a `Series` from a `dict` containing `NaN` as key ([GH18480](https://github.com/pandas-dev/pandas/issues/18480))  
- Fixed construction of a `DataFrame` from a `dict` containing `NaN` as key ([GH18455](https://github.com/pandas-dev/pandas/issues/18455))  
- Disabled construction of a `Series` where len(index) > len(data) = 1, which previously would broadcast the data item, and now raises a `ValueError` ([GH18819](https://github.com/pandas-dev/pandas/issues/18819))  
- Suppressed error in the construction of a `DataFrame` from a `dict` containing scalar values when the corresponding keys are not included in the passed index ([GH18600](https://github.com/pandas-dev/pandas/issues/18600))  
- Fixed (changed from `object` to `float64`) dtype of `DataFrame` initialized with axes, no data, and `dtype=int` ([GH19646](https://github.com/pandas-dev/pandas/issues/19646))  
- Bug in `Series.rank()` where `Series` containing `NaT` modifies the `Series` inplace ([GH18521](https://github.com/pandas-dev/pandas/issues/18521))  
- Bug in `cut()` which fails when using readonly arrays ([GH18773](https://github.com/pandas-dev/pandas/issues/18773))  
- Bug in `DataFrame.pivot_table()` which fails when the `aggfunc` arg is of type string.  The behavior is now consistent with other methods like `agg` and `apply` ([GH18713](https://github.com/pandas-dev/pandas/issues/18713))  
- Bug in `DataFrame.merge()` in which merging using `Index` objects as vectors raised an Exception ([GH19038](https://github.com/pandas-dev/pandas/issues/19038))  
- Bug in `DataFrame.stack()`, `DataFrame.unstack()`, `Series.unstack()` which were not returning subclasses ([GH15563](https://github.com/pandas-dev/pandas/issues/15563))  
- Bug in timezone comparisons, manifesting as a conversion of the index to UTC in `.concat()` ([GH18523](https://github.com/pandas-dev/pandas/issues/18523))  
- Bug in `concat()` when concatenating sparse and dense series it returns only a `SparseDataFrame`. Should be a `DataFrame`. ([GH18914](https://github.com/pandas-dev/pandas/issues/18914), [GH18686](https://github.com/pandas-dev/pandas/issues/18686), and [GH16874](https://github.com/pandas-dev/pandas/issues/16874))  
- Improved error message for `DataFrame.merge()` when there is no common merge key ([GH19427](https://github.com/pandas-dev/pandas/issues/19427))  
- Bug in `DataFrame.join()` which does an `outer` instead of a `left` join when being called with multiple DataFrames and some have non-unique indices ([GH19624](https://github.com/pandas-dev/pandas/issues/19624))  
- `Series.rename()` now accepts `axis` as a kwarg ([GH18589](https://github.com/pandas-dev/pandas/issues/18589))  
- Bug in `rename()` where an Index of same-length tuples was converted to a MultiIndex ([GH19497](https://github.com/pandas-dev/pandas/issues/19497))  
- Comparisons between `Series` and `Index` would return a `Series` with an incorrect name, ignoring the `Index`’s name attribute ([GH19582](https://github.com/pandas-dev/pandas/issues/19582))  
- Bug in `qcut()` where datetime and timedelta data with `NaT` present raised a `ValueError` ([GH19768](https://github.com/pandas-dev/pandas/issues/19768))  
- Bug in `DataFrame.iterrows()`, which would infers strings not compliant to [ISO8601](https://en.wikipedia.org/wiki/ISO_8601) to datetimes ([GH19671](https://github.com/pandas-dev/pandas/issues/19671))  
- Bug in `Series` constructor with `Categorical` where a `ValueError` is not raised when an index of different length is given ([GH19342](https://github.com/pandas-dev/pandas/issues/19342))  
- Bug in `DataFrame.astype()` where column metadata is lost when converting to categorical or a dictionary of dtypes ([GH19920](https://github.com/pandas-dev/pandas/issues/19920))  
- Bug in `cut()` and `qcut()` where timezone information was dropped ([GH19872](https://github.com/pandas-dev/pandas/issues/19872))  
- Bug in `Series` constructor with a `dtype=str`, previously raised in some cases ([GH19853](https://github.com/pandas-dev/pandas/issues/19853))  
- Bug in `get_dummies()`, and `select_dtypes()`, where duplicate column names caused incorrect behavior ([GH20848](https://github.com/pandas-dev/pandas/issues/20848))  
- Bug in `isna()`, which cannot handle ambiguous typed lists ([GH20675](https://github.com/pandas-dev/pandas/issues/20675))  
- Bug in `concat()` which raises an error when concatenating TZ-aware dataframes and all-NaT dataframes ([GH12396](https://github.com/pandas-dev/pandas/issues/12396))  
- Bug in `concat()` which raises an error when concatenating empty TZ-aware series ([GH18447](https://github.com/pandas-dev/pandas/issues/18447))  

### Other

- Improved error message when attempting to use a Python keyword as an identifier in a `numexpr` backed query ([GH18221](https://github.com/pandas-dev/pandas/issues/18221))  
- Bug in accessing a `pandas.get_option()`, which raised `KeyError` rather than `OptionError` when looking up a non-existent option key in some cases ([GH19789](https://github.com/pandas-dev/pandas/issues/19789))  
- Bug in `testing.assert_series_equal()` and `testing.assert_frame_equal()` for Series or DataFrames with differing unicode data ([GH20503](https://github.com/pandas-dev/pandas/issues/20503))  



<a id='whatsnew-0-23-0-contributors'></a>

## Contributors

A total of 328 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Aaron Critchley  
- AbdealiJK +  
- Adam Hooper +  
- Albert Villanova del Moral  
- Alejandro Giacometti +  
- Alejandro Hohmann +  
- Alex Rychyk  
- Alexander Buchkovsky  
- Alexander Lenail +  
- Alexander Michael Schade  
- Aly Sivji +  
- Andreas Költringer +  
- Andrew  
- Andrew Bui +  
- András Novoszáth +  
- Andy Craze +  
- Andy R. Terrel  
- Anh Le +  
- Anil Kumar Pallekonda +  
- Antoine Pitrou +  
- Antonio Linde +  
- Antonio Molina +  
- Antonio Quinonez +  
- Armin Varshokar +  
- Artem Bogachev +  
- Avi Sen +  
- Azeez Oluwafemi +  
- Ben Auffarth +  
- Bernhard Thiel +  
- Bhavesh Poddar +  
- BielStela +  
- Blair +  
- Bob Haffner  
- Brett Naul +  
- Brock Mendel  
- Bryce Guinta +  
- Carlos Eduardo Moreira dos Santos +  
- Carlos García Márquez +  
- Carol Willing  
- Cheuk Ting Ho +  
- Chitrank Dixit +  
- Chris  
- Chris Burr +  
- Chris Catalfo +  
- Chris Mazzullo  
- Christian Chwala +  
- Cihan Ceyhan +  
- Clemens Brunner  
- Colin +  
- Cornelius Riemenschneider  
- Crystal Gong +  
- DaanVanHauwermeiren  
- Dan Dixey +  
- Daniel Frank +  
- Daniel Garrido +  
- Daniel Sakuma +  
- DataOmbudsman +  
- Dave Hirschfeld  
- Dave Lewis +  
- David Adrián Cañones Castellano +  
- David Arcos +  
- David C Hall +  
- David Fischer  
- David Hoese +  
- David Lutz +  
- David Polo +  
- David Stansby  
- Dennis Kamau +  
- Dillon Niederhut  
- Dimitri +  
- Dr. Irv  
- Dror Atariah  
- Eric Chea +  
- Eric Kisslinger  
- Eric O. LEBIGOT (EOL) +  
- FAN-GOD +  
- Fabian Retkowski +  
- Fer Sar +  
- Gabriel de Maeztu +  
- Gianpaolo Macario +  
- Giftlin Rajaiah  
- Gilberto Olimpio +  
- Gina +  
- Gjelt +  
- Graham Inggs +  
- Grant Roch  
- Grant Smith +  
- Grzegorz Konefał +  
- Guilherme Beltramini  
- HagaiHargil +  
- Hamish Pitkeathly +  
- Hammad Mashkoor +  
- Hannah Ferchland +  
- Hans  
- Haochen Wu +  
- Hissashi Rocha +  
- Iain Barr +  
- Ibrahim Sharaf ElDen +  
- Ignasi Fosch +  
- Igor Conrado Alves de Lima +  
- Igor Shelvinskyi +  
- Imanflow +  
- Ingolf Becker  
- Israel Saeta Pérez  
- Iva Koevska +  
- Jakub Nowacki +  
- Jan F-F +  
- Jan Koch +  
- Jan Werkmann  
- Janelle Zoutkamp +  
- Jason Bandlow +  
- Jaume Bonet +  
- Jay Alammar +  
- Jeff Reback  
- JennaVergeynst  
- Jimmy Woo +  
- Jing Qiang Goh +  
- Joachim Wagner +  
- Joan Martin Miralles +  
- Joel Nothman  
- Joeun Park +  
- John Cant +  
- Johnny Metz +  
- Jon Mease  
- Jonas Schulze +  
- Jongwony +  
- Jordi Contestí +  
- Joris Van den Bossche  
- José F. R. Fonseca +  
- Jovixe +  
- Julio Martinez +  
- Jörg Döpfert  
- KOBAYASHI Ittoku +  
- Kate Surta +  
- Kenneth +  
- Kevin Kuhl  
- Kevin Sheppard  
- Krzysztof Chomski  
- Ksenia +  
- Ksenia Bobrova +  
- Kunal Gosar +  
- Kurtis Kerstein +  
- Kyle Barron +  
- Laksh Arora +  
- Laurens Geffert +  
- Leif Walsh  
- Liam Marshall +  
- Liam3851 +  
- Licht Takeuchi  
- Liudmila +  
- Ludovico Russo +  
- Mabel Villalba +  
- Manan Pal Singh +  
- Manraj Singh  
- Marc +  
- Marc Garcia  
- Marco Hemken +  
- Maria del Mar Bibiloni +  
- Mario Corchero +  
- Mark Woodbridge +  
- Martin Journois +  
- Mason Gallo +  
- Matias Heikkilä +  
- Matt Braymer-Hayes  
- Matt Kirk +  
- Matt Maybeno +  
- Matthew Kirk +  
- Matthew Rocklin +  
- Matthew Roeschke  
- Matthias Bussonnier +  
- Max Mikhaylov +  
- Maxim Veksler +  
- Maximilian Roos  
- Maximiliano Greco +  
- Michael Penkov  
- Michael Röttger +  
- Michael Selik +  
- Michael Waskom  
- Mie~~~  
- Mike Kutzma +  
- Ming Li +  
- Mitar +  
- Mitch Negus +  
- Montana Low +  
- Moritz Münst +  
- Mortada Mehyar  
- Myles Braithwaite +  
- Nate Yoder  
- Nicholas Ursa +  
- Nick Chmura  
- Nikos Karagiannakis +  
- Nipun Sadvilkar +  
- Nis Martensen +  
- Noah +  
- Noémi Éltető +  
- Olivier Bilodeau +  
- Ondrej Kokes +  
- Onno Eberhard +  
- Paul Ganssle +  
- Paul Mannino +  
- Paul Reidy  
- Paulo Roberto de Oliveira Castro +  
- Pepe Flores +  
- Peter Hoffmann  
- Phil Ngo +  
- Pietro Battiston  
- Pranav Suri +  
- Priyanka Ojha +  
- Pulkit Maloo +  
- README Bot +  
- Ray Bell +  
- Riccardo Magliocchetti +  
- Ridhwan Luthra +  
- Robert Meyer  
- Robin  
- Robin Kiplang’at +  
- Rohan Pandit +  
- Rok Mihevc +  
- Rouz Azari  
- Ryszard T. Kaleta +  
- Sam Cohan  
- Sam Foo  
- Samir Musali +  
- Samuel Sinayoko +  
- Sangwoong Yoon  
- SarahJessica +  
- Sharad Vijalapuram +  
- Shubham Chaudhary +  
- SiYoungOh +  
- Sietse Brouwer  
- Simone Basso +  
- Stefania Delprete +  
- Stefano Cianciulli +  
- Stephen Childs +  
- StephenVoland +  
- Stijn Van Hoey +  
- Sven  
- Talitha Pumar +  
- Tarbo Fukazawa +  
- Ted Petrou +  
- Thomas A Caswell  
- Tim Hoffmann +  
- Tim Swast  
- Tom Augspurger  
- Tommy +  
- Tulio Casagrande +  
- Tushar Gupta +  
- Tushar Mittal +  
- Upkar Lidder +  
- Victor Villas +  
- Vince W +  
- Vinícius Figueiredo +  
- Vipin Kumar +  
- WBare  
- Wenhuan +  
- Wes Turner  
- William Ayd  
- Wilson Lin +  
- Xbar  
- Yaroslav Halchenko  
- Yee Mey  
- Yeongseon Choe +  
- Yian +  
- Yimeng Zhang  
- ZhuBaohe +  
- Zihao Zhao +  
- adatasetaday +  
- akielbowicz +  
- akosel +  
- alinde1 +  
- amuta +  
- bolkedebruin  
- cbertinato  
- cgohlke  
- charlie0389 +  
- chris-b1  
- csfarkas +  
- dajcs +  
- deflatSOCO +  
- derestle-htwg  
- discort  
- dmanikowski-reef +  
- donK23 +  
- elrubio +  
- fivemok +  
- fjdiod  
- fjetter +  
- froessler +  
- gabrielclow  
- gfyoung  
- ghasemnaddaf  
- h-vetinari +  
- himanshu awasthi +  
- ignamv +  
- jayfoad +  
- jazzmuesli +  
- jbrockmendel  
- jen w +  
- jjames34 +  
- joaoavf +  
- joders +  
- jschendel  
- juan huguet +  
- l736x +  
- luzpaz +  
- mdeboc +  
- miguelmorin +  
- miker985  
- miquelcamprodon +  
- orereta +  
- ottiP +  
- peterpanmj +  
- rafarui +  
- raph-m +  
- readyready15728 +  
- rmihael +  
- samghelms +  
- scriptomation +  
- sfoo +  
- stefansimik +  
- stonebig  
- tmnhat2001 +  
- tomneep +  
- topper-123  
- tv3141 +  
- verakai +  
- xpvpc +  
- zhanghui +  